In [2]:
import pandas as pd
import requests
import json
import re
import sys
import numpy as np
import os
import nltk
import pickle
from sklearn.utils import shuffle

sys.path.append('../src')

from commons import elastic
from langdetect import detect
from text_processing import text_normalizer
from importlib import reload
from text_processing import duplicate_finder
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
text_normalizer = reload(text_normalizer)
from dateutil.parser import parse
from utilities import excel_writer, excel_reader
from text_processing import abbreviations_resolver
abbreviations_resolver = reload(abbreviations_resolver)
from utilities import utils

stemmer = SnowballStemmer("english")
lmtzr = WordNetLemmatizer()
#nlp = spacy.load('en_core_web_sm')

nlp = spacy.load("en_core_web_md")

filter_word_list = text_normalizer.build_filter_dictionary(["../data/Filter_Geo_Names.xlsx"])

_abbreviations_resolver = abbreviations_resolver.AbbreviationsResolver(filter_word_list)
_abbreviations_resolver.load_model("../model/abbreviations_dicts")

C:\Users\Maryia_Ivanina\Anaconda3\envs\cornell\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Download data and extract from XML

In [12]:
folder = r"C:\Users\Maryia_Ivanina\cornell_project\CoSAICornell"

In [ ]:
import zipfile
for i in range(0, 41):
    path_to_zip_file = os.path.join(folder, "subfolder-%d.zip"%i)
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(os.path.join(folder, "subfolder-%d"%i))

In [15]:
from bs4 import BeautifulSoup
content = None 
with open(os.path.join(folder, "subfolder-4", "20043190786.xml"), "rb") as f:
    content = BeautifulSoup(f.read(), "lxml")

In [13]:
def prepare_initial_dataset(folder):
    df = pd.DataFrame(data= {'article_name': os.listdir(folder)})
    df["abstract"] = ""
    df["title"] = ""
    df["keywords"] = ""
    df["authors"] = ""
    df["journal_name"] = ""
    df["publisher"] = ""
    df["publisher_location"] = ""
    df["year"] = ""
    df["url"] = ""
    df["affiliation"] = ""
    df["corporate_author"] = ""
    df["article_id"] = ""
    df["cabi_subjects"] = ""
    df["language"] = ""
    df["email"] = ""
    df["first_author_affiliation"] = ""
    return df

def fill_text(content, tag_name):
    if content.find(tag_name):
        return content.find(tag_name).text.strip()
    return ""

def fill_abstract(content):
    tag_name = "abstract"
    info = []
    for tag in content.find_all(tag_name):
        if tag.find("p"):
            info.extend([t.text.strip() for t in tag.find_all("p")])
        else:
            info.append(tag.text)
    return "\n".join(info)

def fill_tags(content, tag_name):
    if content.find_all(tag_name):
        return ";".join([tag.text.strip() for tag in content.find_all(tag_name)])
    return ""

def fill_authors(content):
    tag_name = "contrib"
    authors = []
    affiliations = []
    emails = []
    if content.find_all(tag_name):
        for tag in content.find_all(tag_name):
            if tag["contrib-type"] == "author":
                authors.append(fill_tags(tag, "string-name"))
                if tag.find("aff"):
                    address_parts = []
                    for ch in tag.find("aff").children:
                        try:
                            address_parts.append(ch.text.strip().replace(";", ","))
                        except:
                            pass
                    affiliations.append(", ".join(address_parts))
                if tag.find("email"):
                    emails.append(fill_tags(tag, "email"))
    return ";".join(authors), ";".join(affiliations), ";".join(emails)

def fill_author_affiliations(content):
    tag_name = "contrib"
    authors = []
    if content.find_all(tag_name):
        for tag in content.find_all(tag_name):
            if tag["contrib-type"] == "corporateauthor":
                string_name = fill_tags(tag, "string-name")
                if tag.find("aff"):
                    address_parts = []
                    for ch in tag.find("aff").children:
                        try:
                            address_parts.append(ch.text.strip().replace(";", ","))
                        except:
                            pass
                    string_name = string_name + ", " + ", ".join(address_parts)
                authors.append(string_name)
    return ";".join(authors)

def fill_article_id(content, pub_type):
    for tag in content.find_all("article-id"):
        try:
            if tag["pub-id-type"] == pub_type:
                return tag.text
        except:
            pass
    return ""

def fill_cabi_tags(content):
    tag_name = "subject"
    if content.find_all(tag_name):
        tags = []
        languages = []
        for tag in content.find_all(tag_name):
            try:
                if tag["vocab"] == "language":
                    languages.append(tag.text.strip())
                else:
                    tags.append(tag.text.strip())
            except:
                pass
        return ";".join(tags), ";".join(languages)
    return "", ""

def extract_keywords(abstract):
    abstract_parts = abstract.split("KEYWORDS:")
    if len(abstract_parts) == 1:
        return abstract, ""
    abstract, keywords = abstract_parts[0].strip(), " | ".join(abstract_parts[1:])
    keywords = ";".join([k.replace(".", " ").strip() for k in keywords.split("|")])
    return abstract, keywords

def fill_fields(content, df, i):
    df["title"].values[i] = fill_text(content, "article-title")
    df["publisher"].values[i] = fill_text(content, "publisher-name")
    df["publisher_location"].values[i] = fill_text(content, "publisher-loc").strip().replace("\n", ";")
    df["article_id"].values[i] = fill_article_id(content, "CABI-pan")
    df["url"].values[i] = fill_article_id(content, "doi")
    if df["url"].values[i].strip():
        df["url"].values[i] = "https://www.doi.org/" + df["url"].values[i]
    df["cabi_subjects"].values[i], df["language"].values[i]  = fill_cabi_tags(content)
    df["year"].values[i] = fill_text(content, "year")
    try:
        df["year"].values[i] = int(df["year"].values[i])
    except:
        print("Year error: '%s' for %d row" % (df["year"].values[i], i))
    df["abstract"].values[i] = fill_abstract(content)
    df["keywords"].values[i] = fill_tags(content, "kwd")
    abstract, keywords_from_abstract = extract_keywords(df["abstract"].values[i])
    df["abstract"].values[i] = abstract
    if keywords_from_abstract.strip():
        df["keywords"].values[i] = df["keywords"].values[i] + ";" + keywords_from_abstract
    df["authors"].values[i], df["affiliation"].values[i], df["email"].values[i] = fill_authors(content)
    df["corporate_author"].values[i] = fill_author_affiliations(content)
    df["journal_name"].values[i] = fill_text(content, "journal-title")
    df["first_author_affiliation"].values[i] = fill_first_author_affiliations(content)
    return df

def fill_first_author_affiliations(content):
    tag_name = "contrib"
    affiliation = ""
    if content.find_all(tag_name):
        for tag in content.find_all(tag_name):
            if tag["contrib-type"] == "author":
                string_name = ""
                if tag.find("aff"):
                    address_parts = []
                    for ch in tag.find("aff").children:
                        try:
                            address_parts.append(ch.text.strip().replace(";", ","))
                        except:
                            pass
                    string_name = ", ".join(address_parts)
                affiliation = string_name
            if affiliation.strip():
                break
    return affiliation

def prepare_dataset(folder, folder_to_save):
    os.makedirs(folder_to_save, exist_ok=True)
    df = prepare_initial_dataset(folder)
    for i in range(len(df)):
        with open(os.path.join(folder, df["article_name"].values[i]), encoding="utf-8") as f:
            try:
                meta_doc = BeautifulSoup(f.read(), "lxml")
                df = fill_fields(meta_doc, df, i)
            except Exception as e:
                print(e)
                print(df["article_name"].values[i]," ### ", i)
    if len(df) > 0:
        excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_to_save, os.path.basename(folder) + ".xlsx"))

In [16]:
from time import time
for i in range(10, 11):
    sub_folder = os.path.join(folder, "subfolder-%d"%i)
    if os.path.exists(sub_folder):
        print(sub_folder)
        t_start = time()
        prepare_dataset(sub_folder, "../tmp/cosai_data")
        print("Time spent ", time() - t_start)

C:\Users\Maryia_Ivanina\cornell_project\CoSAICornell\subfolder-10
Saving...
Saved to ../tmp/cosai_data\subfolder-10.xlsx
Time spent  261.0172607898712


In [113]:
cnts = [0, 0, 0, 0, 0]
for i in range(0, 41):
    df = excel_reader.ExcelReader().read_df_from_excel("../tmp/cosai_data/subfolder-%d.xlsx"%i)
    cnts[0] += len(df)
    for i in range(len(df)):
        if df["affiliation"].values[i].strip():
            cnts[1] += 1
        if df["email"].values[i].strip():
            cnts[2] += 1
        if df["affiliation"].values[i].strip() or df["email"].values[i].strip():
            cnts[3] += 1
        if df["first_author_affiliation"].values[i].strip():
            cnts[4] += 1
cnts

Read file ../tmp/cosai_data/subfolder-0.xlsx: 5.38s
Processed file ../tmp/cosai_data/subfolder-0.xlsx: 2.62s
Read file ../tmp/cosai_data/subfolder-1.xlsx: 4.33s
Processed file ../tmp/cosai_data/subfolder-1.xlsx: 2.60s
Read file ../tmp/cosai_data/subfolder-2.xlsx: 4.11s
Processed file ../tmp/cosai_data/subfolder-2.xlsx: 2.57s
Read file ../tmp/cosai_data/subfolder-3.xlsx: 4.10s
Processed file ../tmp/cosai_data/subfolder-3.xlsx: 2.36s
Read file ../tmp/cosai_data/subfolder-4.xlsx: 5.19s
Processed file ../tmp/cosai_data/subfolder-4.xlsx: 2.64s
Read file ../tmp/cosai_data/subfolder-5.xlsx: 4.17s
Processed file ../tmp/cosai_data/subfolder-5.xlsx: 2.46s
Read file ../tmp/cosai_data/subfolder-6.xlsx: 3.86s
Processed file ../tmp/cosai_data/subfolder-6.xlsx: 2.69s
Read file ../tmp/cosai_data/subfolder-7.xlsx: 5.43s
Processed file ../tmp/cosai_data/subfolder-7.xlsx: 2.70s
Read file ../tmp/cosai_data/subfolder-8.xlsx: 4.55s
Processed file ../tmp/cosai_data/subfolder-8.xlsx: 2.39s
Read file ../tmp/co

[1225741, 1153004, 933880, 1161919, 1153004]

In [108]:
from text_processing import search_engine_insensitive_to_spelling
search_engine_insensitive_to_spelling = reload(search_engine_insensitive_to_spelling)
from time import time
from interventions_labeling_lib import hyponym_search

folder_to_save = "../tmp/cosai_hyponyms"
folder_with_excel = "../tmp/cosai_data"

for file in os.listdir(folder_with_excel):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_excel, file))
    file_part = file.split(".")[0]
    t_start = time()
    search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
        load_abbreviations = True)
    if os.path.exists("../model/cosai_index/%s" % file_part):
        search_engine_inverted_index.load_model("../model/cosai_index/%s" % file_part)
    else:
        search_engine_inverted_index.create_inverted_index(df)
        search_engine_inverted_index.save_model("../model/cosai_index/%s" % file_part)
    print("Time spent indexing: ", time() - t_start)
    
    file_to_save = os.path.join(folder_to_save, "hyponyms_found_%s.pickle"%file_part)
    os.makedirs(folder_to_save, exist_ok=True)
    if not os.path.exists(file_to_save):
        t_start = time()
        hyponyms_search = hyponym_search.HyponymsSearch()
        hyponyms_search.find_hyponyms_and_hypernyms(
                df, search_engine_inverted_index, file_to_save,
                columns_to_use=["title", "abstract"])
        pickle.dump(hyponyms_search, open(file_to_save,"wb"))
        print("Time spent hyponyms: ", time() - t_start)

Read file ../tmp/cosai_data\subfolder-0.xlsx: 5.92s
Processed file ../tmp/cosai_data\subfolder-0.xlsx: 3.64s
Time spent indexing:  6.564329624176025
Read file ../tmp/cosai_data\subfolder-1.xlsx: 4.15s
Processed file ../tmp/cosai_data\subfolder-1.xlsx: 3.12s
Time spent indexing:  5.8737406730651855
Read file ../tmp/cosai_data\subfolder-10.xlsx: 5.08s
Processed file ../tmp/cosai_data\subfolder-10.xlsx: 3.32s
Time spent indexing:  6.8321239948272705
Read file ../tmp/cosai_data\subfolder-11.xlsx: 5.73s
Processed file ../tmp/cosai_data\subfolder-11.xlsx: 3.43s
Time spent indexing:  8.33012342453003
Read file ../tmp/cosai_data\subfolder-12.xlsx: 4.57s
Processed file ../tmp/cosai_data\subfolder-12.xlsx: 3.11s
Time spent indexing:  7.6806159019470215
Read file ../tmp/cosai_data\subfolder-13.xlsx: 6.26s
Processed file ../tmp/cosai_data\subfolder-13.xlsx: 3.12s
Time spent indexing:  6.625067472457886
Read file ../tmp/cosai_data\subfolder-14.xlsx: 5.64s
Processed file ../tmp/cosai_data\subfolder-

Processed 0 abbreviations
Processed 3000 abbreviations
Processed 6000 abbreviations
Processed 9000 abbreviations
Processed 12000 abbreviations
Processed 15000 abbreviations
Processed 18000 abbreviations
Processed 21000 abbreviations
Processed 24000 abbreviations
Processed 27000 abbreviations
Processed 27129 abbreviations
Time spent indexing:  874.1564021110535
Processed 0 articles
Processed 5000 articles
Processed 10000 articles
Processed 15000 articles
Processed 20000 articles
Processed 25000 articles
Processed 29999 articles
merge concepts: 69 s
groups to merge: 19354
Time spent hyponyms:  39982.425367832184
Read file ../tmp/cosai_data\subfolder-37.xlsx: 5.58s
Processed file ../tmp/cosai_data\subfolder-37.xlsx: 2.56s
Processed 0 articles
Processed 20000 articles
Processed 29999 articles
Processed 0 abbreviations
Processed 3000 abbreviations
Processed 6000 abbreviations
Processed 9000 abbreviations
Processed 12000 abbreviations
Processed 15000 abbreviations
Processed 18000 abbreviatio

In [6]:
from interventions_labeling_lib import hyponym_search, hyponym_statistics
from text_processing import search_engine_insensitive_to_spelling
search_engine_insensitive_to_spelling = reload(search_engine_insensitive_to_spelling)
from time import time


filter_word_list = text_normalizer.build_filter_dictionary(["../data/Filter_Geo_Names.xlsx"])
folder_with_excel = "../tmp/cosai_data"
key_word_mappings = ["practice", "approach", "intervention", "input",
"strategy", "policy", "program", "programme", "initiative", "technology",
 "science", "technique", "innovation", "biotechnology","machine", "mechanism", 
 "equipment","tractor","device","machinery","project","gadget"]
filter_hypernyms =  ["practice", "approach", "intervention", 
    "input","strategy", "policy", "program", "programme", "initiative", "technology", \
    "science", "technique", "innovation", "biotechnology","machine", "mechanism", 
    "equipment","device","project"]

folder_to_save = "../tmp/cosai_hyponyms_interv_files"
os.makedirs(folder_to_save, exist_ok=True)

_abbreviation_resolver = abbreviations_resolver.AbbreviationsResolver(filter_word_list)
_abbreviation_resolver.load_model("../model/abbreviations_dicts")

for file in os.listdir(folder_with_excel):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_excel, file))
    file_part = file.split(".")[0]
    t_start = time()
    search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
        load_abbreviations = True)
    if os.path.exists("../model/cosai_index/%s" % file_part):
        search_engine_inverted_index.load_model("../model/cosai_index/%s" % file_part)
    else:
        search_engine_inverted_index.create_inverted_index(df)
        search_engine_inverted_index.save_model("../model/cosai_index/%s" % file_part)
    print("Time spent indexing: ", time() - t_start)
    
    hyponyms_search = pickle.load(open(os.path.join("../tmp/cosai_hyponyms", "hyponyms_found_%s.pickle"%file_part), "rb"))
    hs_pruned = hyponym_statistics.HyponymStatistics(key_word_mappings, search_engine_inverted_index,
                                                     _abbreviation_resolver, hyponyms_search.dict_hyponyms,hyponyms_search,\
                                                     filter_word_list=filter_word_list, filter_hypernyms = filter_hypernyms)
    hs_pruned.save_pruned_hyponym_pairs_to_file(
        os.path.join(folder_to_save, "hyponyms_found_cleaned_%s.xlsx" % file_part), folder_to_save)
    print("All time spent ", time() - t_start)

Read file ../tmp/cosai_data\subfolder-0.xlsx: 3.90s
Processed file ../tmp/cosai_data\subfolder-0.xlsx: 3.02s
Time spent indexing:  5.712380409240723
Saving...
Saved to ../tmp/cosai_hyponyms_interv_files\cleaned_words 1630563787.xlsx
Saving...
Saved to ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-0.xlsx
All time spent  45.11997985839844
Read file ../tmp/cosai_data\subfolder-1.xlsx: 3.97s
Processed file ../tmp/cosai_data\subfolder-1.xlsx: 3.24s
Time spent indexing:  5.5128161907196045
Saving...
Saved to ../tmp/cosai_hyponyms_interv_files\cleaned_words 1630563840.xlsx
Saving...
Saved to ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-1.xlsx
All time spent  45.07873320579529
Read file ../tmp/cosai_data\subfolder-10.xlsx: 4.76s
Processed file ../tmp/cosai_data\subfolder-10.xlsx: 3.86s
Time spent indexing:  6.044091463088989
Saving...
Saved to ../tmp/cosai_hyponyms_interv_files\cleaned_words 1630563898.xlsx
Saving...
Saved to ../tmp/cosai_hyponyms_i

Saved to ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-29.xlsx
All time spent  71.94015526771545
Read file ../tmp/cosai_data\subfolder-3.xlsx: 6.01s
Processed file ../tmp/cosai_data\subfolder-3.xlsx: 3.99s
Time spent indexing:  5.876176834106445
Saving...
Saved to ../tmp/cosai_hyponyms_interv_files\cleaned_words 1630565401.xlsx
Saving...
Saved to ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-3.xlsx
All time spent  50.93112134933472
Read file ../tmp/cosai_data\subfolder-30.xlsx: 6.86s
Processed file ../tmp/cosai_data\subfolder-30.xlsx: 3.92s
Time spent indexing:  8.040663957595825
Saving...
Saved to ../tmp/cosai_hyponyms_interv_files\cleaned_words 1630565485.xlsx
Saving...
Saved to ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-30.xlsx
All time spent  73.48305487632751
Read file ../tmp/cosai_data\subfolder-31.xlsx: 5.63s
Processed file ../tmp/cosai_data\subfolder-31.xlsx: 4.41s
Time spent indexing:  8.679456949234009
Savin

In [9]:
import pandas as pd
import gensim
import os
import numpy as np
from importlib import reload
from interventions_labeling_lib import intervention_labeling
from interventions_labeling_lib.intervention_labels import InterventionLabels
from utilities import excel_reader, excel_writer
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix


folder_with_labeled = "../tmp/cosai_interv_additional_labeled"
folder_with_interv = "../tmp/cosai_hyponyms_interv_files"
intervention_labeler = intervention_labeling.InterventionLabeling(google_models_folder="../model/synonyms_retrained_new")
intervention_labeler.load_previous_models("../model/intervention_labels_model")
os.makedirs(folder_with_labeled, exist_ok=True)

for file in os.listdir(folder_with_interv):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_interv, file))
    df_pred = intervention_labeler.predict_class(df.values)
    df["Label"] = [InterventionLabels.INTERVENTION_NUMBER_TO_LABEL[label] for label in df_pred]
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_with_labeled, file), column_interventions=['Label'],
                                           dict_with_colors=InterventionLabels.INTERVENTION_LABELS_COLOR)

Google word2vec loaded
Fast text loaded
Phrases loaded
Read file ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-0.xlsx: 0.11s
Processed file ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-0.xlsx: 0.05s


../src\interventions_labeling_lib\intervention_labeling.py:172: RuntimeWarning: invalid value encountered in true_divide
  res = x / np.sum(x, axis=1)[:, None]


Saving...
Saved to ../tmp/cosai_interv_additional_labeled\hyponyms_found_cleaned_subfolder-0.xlsx
Read file ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-1.xlsx: 0.08s
Processed file ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-1.xlsx: 0.05s


../src\interventions_labeling_lib\intervention_labeling.py:172: RuntimeWarning: invalid value encountered in true_divide
  res = x / np.sum(x, axis=1)[:, None]


Saving...
Saved to ../tmp/cosai_interv_additional_labeled\hyponyms_found_cleaned_subfolder-1.xlsx
Read file ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-10.xlsx: 0.08s
Processed file ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-10.xlsx: 0.05s


...

Saving...
Saved to ../tmp/cosai_interv_additional_labeled\hyponyms_found_cleaned_subfolder-7.xlsx
Read file ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-8.xlsx: 0.25s
Processed file ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-8.xlsx: 0.12s


../src\interventions_labeling_lib\intervention_labeling.py:172: RuntimeWarning: invalid value encountered in true_divide
  res = x / np.sum(x, axis=1)[:, None]


Saving...
Saved to ../tmp/cosai_interv_additional_labeled\hyponyms_found_cleaned_subfolder-8.xlsx
Read file ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-9.xlsx: 0.11s
Processed file ../tmp/cosai_hyponyms_interv_files\hyponyms_found_cleaned_subfolder-9.xlsx: 0.06s


../src\interventions_labeling_lib\intervention_labeling.py:172: RuntimeWarning: invalid value encountered in true_divide
  res = x / np.sum(x, axis=1)[:, None]


Saving...
Saved to ../tmp/cosai_interv_additional_labeled\hyponyms_found_cleaned_subfolder-9.xlsx


# Deduplication

In [3]:
folder_with_cosai_by_year = "../tmp/cosai_by_year"
os.makedirs(folder_with_cosai_by_year, exist_ok=True)
all_dfs_by_year = {}
for file in os.listdir("../tmp/cosai_data"):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/cosai_data", file))
    for key_ in df["year"].value_counts().keys():
        if key_ not in all_dfs_by_year:
            all_dfs_by_year[key_] = pd.DataFrame()
        all_dfs_by_year[key_] = pd.concat([all_dfs_by_year[key_], df[df["year"] == key_]], axis=0)

Read file ../tmp/cosai_data\subfolder-0.xlsx: 10.33s
Processed file ../tmp/cosai_data\subfolder-0.xlsx: 7.16s
Read file ../tmp/cosai_data\subfolder-1.xlsx: 10.01s
Processed file ../tmp/cosai_data\subfolder-1.xlsx: 6.82s
Read file ../tmp/cosai_data\subfolder-10.xlsx: 10.66s
Processed file ../tmp/cosai_data\subfolder-10.xlsx: 6.95s
Read file ../tmp/cosai_data\subfolder-11.xlsx: 11.73s
Processed file ../tmp/cosai_data\subfolder-11.xlsx: 7.07s
Read file ../tmp/cosai_data\subfolder-12.xlsx: 10.65s
Processed file ../tmp/cosai_data\subfolder-12.xlsx: 6.98s
Read file ../tmp/cosai_data\subfolder-13.xlsx: 10.43s
Processed file ../tmp/cosai_data\subfolder-13.xlsx: 7.23s
Read file ../tmp/cosai_data\subfolder-14.xlsx: 11.41s
Processed file ../tmp/cosai_data\subfolder-14.xlsx: 7.63s
Read file ../tmp/cosai_data\subfolder-15.xlsx: 12.27s
Processed file ../tmp/cosai_data\subfolder-15.xlsx: 7.08s
Read file ../tmp/cosai_data\subfolder-16.xlsx: 11.24s
Processed file ../tmp/cosai_data\subfolder-16.xlsx: 6.

In [4]:
for year in all_dfs_by_year:
    excel_writer.ExcelWriter().save_df_in_excel(all_dfs_by_year[year], os.path.join(folder_with_cosai_by_year, "data_year_%d.xlsx" % year))

Saving...
Saved to ../tmp/cosai_by_year\data_year_2000.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2001.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2004.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2005.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2003.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2002.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2009.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2008.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2006.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2007.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2010.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2011.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2012.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2013.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2014.xlsx
Saving...
Saved to ../tmp/cosai_by_year\data_year_2015.xlsx
Saving...
Saved to ../tmp/cosai_by_year\

In [5]:
del all_dfs_by_year

In [6]:
from text_processing import duplicate_finder
duplicate_finder = reload(duplicate_finder)
from time import time

dedupl_folder = "../tmp/processed_deduplicated_cosai"
os.makedirs(dedupl_folder, exist_ok=True)
for file in os.listdir(folder_with_cosai_by_year):
    start_time = time()
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_cosai_by_year, file))
    print("before ", len(df))
    folder_with_deduplicates = os.path.join("../tmp/deduplicated_cosai", file.split(".")[0])
    _duplicate_finder = duplicate_finder.DuplicateFinder(folder_with_deduplicates, additional_columns=["authors", "corporate_author"])
    df = _duplicate_finder.remove_duplicates_in_one_df_by_title(df, [])
    print("after ", len(df))
    print("Spent time: ", time() - start_time)
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(dedupl_folder, file))

Read file ../tmp/cosai_by_year\data_year_2000.xlsx: 8.74s
Processed file ../tmp/cosai_by_year\data_year_2000.xlsx: 6.62s
before  32038
12.495641708374023
104.68890500068665
Year 2000: 23946 items
Checked all articles
Saving...
Saved to ../tmp/deduplicated_cosai\data_year_2000\duplicate_in_one_dataset_1629821282.xlsx
Merged file
Filtered dataset
after  31806
Spent time:  254.2719602584839
Saving...
Saved to ../tmp/processed_deduplicated_cosai\data_year_2000.xlsx
Read file ../tmp/cosai_by_year\data_year_2001.xlsx: 5.60s
Processed file ../tmp/cosai_by_year\data_year_2001.xlsx: 4.07s
before  24811
8.30266547203064
82.00332951545715
Year 2001: 18920 items
Checked all articles
Saving...
Saved to ../tmp/deduplicated_cosai\data_year_2001\duplicate_in_one_dataset_1629821482.xlsx
Merged file
Filtered dataset
after  24577
Spent time:  175.61480450630188
Saving...
Saved to ../tmp/processed_deduplicated_cosai\data_year_2001.xlsx
Read file ../tmp/cosai_by_year\data_year_2002.xlsx: 6.48s
Processed fi

Merged file
Filtered dataset
after  88213
Spent time:  1457.9921002388
Saving...
Saved to ../tmp/processed_deduplicated_cosai\data_year_2017.xlsx
Read file ../tmp/cosai_by_year\data_year_2018.xlsx: 23.03s
Processed file ../tmp/cosai_by_year\data_year_2018.xlsx: 13.58s
before  99653
35.292829513549805
430.9305601119995
Year 2018: 64061 items
Checked all articles
Saving...
Saved to ../tmp/deduplicated_cosai\data_year_2018\duplicate_in_one_dataset_1629834739.xlsx
Merged file
Filtered dataset
after  99445
Spent time:  1710.3780751228333
Saving...
Saved to ../tmp/processed_deduplicated_cosai\data_year_2018.xlsx
Read file ../tmp/cosai_by_year\data_year_2019.xlsx: 24.43s
Processed file ../tmp/cosai_by_year\data_year_2019.xlsx: 15.11s
before  108467
31.162141799926758
405.92951226234436
Year 2019: 68479 items
Checked all articles
Saving...
Saved to ../tmp/deduplicated_cosai\data_year_2019\duplicate_in_one_dataset_1629836553.xlsx
Merged file
Filtered dataset
after  108266
Spent time:  1729.5240

In [7]:
all_data_cnt_before_dedupl = 0
for file in os.listdir(folder_with_cosai_by_year):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_cosai_by_year, file))
    all_data_cnt_before_dedupl += len(df)

all_data_cnt_after_dedupl = 0
for file in os.listdir(dedupl_folder):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(dedupl_folder, file))
    all_data_cnt_after_dedupl += len(df)

print("Before deduplication: %d articles" % all_data_cnt_before_dedupl)
print("After deduplication: %d articles" % all_data_cnt_after_dedupl)

Read file ../tmp/cosai_by_year\data_year_2000.xlsx: 3.70s
Processed file ../tmp/cosai_by_year\data_year_2000.xlsx: 2.46s
Read file ../tmp/cosai_by_year\data_year_2001.xlsx: 2.86s
Processed file ../tmp/cosai_by_year\data_year_2001.xlsx: 1.92s
Read file ../tmp/cosai_by_year\data_year_2002.xlsx: 3.25s
Processed file ../tmp/cosai_by_year\data_year_2002.xlsx: 2.21s
Read file ../tmp/cosai_by_year\data_year_2003.xlsx: 3.39s
Processed file ../tmp/cosai_by_year\data_year_2003.xlsx: 2.28s
Read file ../tmp/cosai_by_year\data_year_2004.xlsx: 3.62s
Processed file ../tmp/cosai_by_year\data_year_2004.xlsx: 2.45s
Read file ../tmp/cosai_by_year\data_year_2005.xlsx: 4.07s
Processed file ../tmp/cosai_by_year\data_year_2005.xlsx: 2.65s
Read file ../tmp/cosai_by_year\data_year_2006.xlsx: 3.98s
Processed file ../tmp/cosai_by_year\data_year_2006.xlsx: 2.60s
Read file ../tmp/cosai_by_year\data_year_2007.xlsx: 4.99s
Processed file ../tmp/cosai_by_year\data_year_2007.xlsx: 3.54s
Read file ../tmp/cosai_by_year\d

# Preparing data for labelling

In [31]:
df = excel_reader.ExcelReader().read_df_from_excel("../tmp/cosai_data/subfolder-0.xlsx")

Read file ../tmp/cosai_data/subfolder-0.xlsx: 4.16s
Processed file ../tmp/cosai_data/subfolder-0.xlsx: 3.10s


In [32]:
from text_processing import search_engine_insensitive_to_spelling

folder_to_save = "../model/search_index_population_df"
small_search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
    load_abbreviations = True, columns_to_process=["title", "abstract"])
if os.path.exists(folder_to_save):
    small_search_engine_inverted_index.load_model(folder_to_save)
else:
    small_search_engine_inverted_index.create_inverted_index(df)
    small_search_engine_inverted_index.save_model(folder_to_save)

In [33]:
from interventions_labeling_lib import interventions_search_for_labeling
interventions_search_for_labeling = reload(interventions_search_for_labeling)
from text_processing import population_tags_finder
population_tags_finder = reload(population_tags_finder)
from text_processing import all_column_filler
all_column_filler = reload(all_column_filler)

_all_column_filler = all_column_filler.AllColumnFiller()
df = _all_column_filler.fill_columns_for_df(df, small_search_engine_inverted_index, _abbreviations_resolver,
                                                settings_json = {"columns":[
   {"column_filler_class":"PopulationTagsFinder", "columns_to_process":["title", "abstract"]},
   {"column_filler_class":"ColumnFiller", "column_name": "sub_populations",
    "column_details": "sub_populations_details", "column_dictionary":"../data/subpopulations.xlsx"},]})

Started processing  {'column_filler_class': 'PopulationTagsFinder', 'columns_to_process': ['title', 'abstract']}
Processed for 2.4300479888916016s
Started processing  {'column_filler_class': 'ColumnFiller', 'column_name': 'sub_populations', 'column_details': 'sub_populations_details', 'column_dictionary': '../data/subpopulations.xlsx'}
High level label: women
High level label: nomadic
High level label: semi-literate
High level label: men
High level label: senior citizens
High level label: youth
High level label: adults
Labelled articles with outcomes: 3063
Processed for 1.7198684215545654s


In [34]:
for i in range(len(df)):
    df["sub_populations_details"].values[i] = ";".join(df["sub_populations_details"].values[i])

In [35]:
df["research_scale"] = ""
for i in range(len(df)):
    if i == 0:
        df["research_scale"].values[i] = ["Small farms", "Medium/Large Scale Farms", 
                                          "Value Chains", "Markets", "Enabling Environment",
                                          "Food Systems", "Others"]
    else:
        df["research_scale"].values[i] = []

In [36]:
df["population_details"] = ""
df["Taken"] = 0
df["interventions_extracted"] = ""

In [37]:
excel_writer.ExcelWriter().save_df_in_excel(df, "all_sentences_population_tags.xlsx")

Saving...
Saved to all_sentences_population_tags.xlsx


In [50]:
chosen_ids = set()
for i in range(len(df)):
    if len(df["sub_populations"].values[i]) > 0 and len(df["population tags"].values[i]) > 0:
        chosen_ids.add(i)
        df["Taken"].values[i] = 1
len(chosen_ids)

311

In [39]:
found_words = {}
for column in ["sub_populations", "population tags"]:
    for i in range(len(df)):
        if df["Taken"].values[i] == 1:
            continue
        for w in df[column].values[i]:
            if w not in found_words:
                found_words[w] = []
            found_words[w].append(i)

In [51]:
for w in found_words:
    print(w, len(found_words[w]))

women 1182
men 966
youth 809
nomadic 29
adults 773
senior citizens 81
semi-literate 5
Farmers 1756
Small scale farmers 478


In [52]:
chosen_ids.update(found_words["nomadic"])
chosen_ids.update(found_words["semi-literate"])
chosen_ids.update(found_words["senior citizens"])
len(chosen_ids)

426

In [53]:
for word in ["women", "Farmers", "Small scale farmers", "men", "youth", "adults"]:
    np.random.shuffle(found_words[word])
    if word in ["adults", "men"]:
        chosen_ids.update(found_words[word][:50])
    else:
        chosen_ids.update(found_words[word][:100])
len(chosen_ids)

915

In [54]:
wo_tags = []
for i in range(len(df)):
    if not df["population tags"].values[i] and not df["sub_populations"].values[i]:
        wo_tags.append(i)
np.random.shuffle(wo_tags)

In [55]:
chosen_ids.update(wo_tags[:85])
len(chosen_ids)

1000

In [56]:
for i in chosen_ids:
    df["Taken"].values[i] = 1

In [57]:
excel_writer.ExcelWriter().save_df_in_excel(df, "all_sentences_population_tags.xlsx")

Saving...
Saved to all_sentences_population_tags.xlsx


In [58]:
excel_writer.ExcelWriter().save_df_in_excel(df.take(list(chosen_ids)), "../tmp/population_tags_data/sample_first.xlsx")

Saving...
Saved to ../tmp/population_tags_data/sample_first.xlsx


In [59]:
excel_writer.ExcelWriter().save_df_in_excel(utils.add_responsible_persons(df.take(list(chosen_ids)).copy()), "../tmp/population_tags_data/sample_first.xlsx")

Saving...
Saved to ../tmp/population_tags_data/sample_first.xlsx


In [60]:
full_df = excel_reader.ExcelReader().read_df_from_excel("../tmp/population_tags_data/sample_first.xlsx")

Read file ../tmp/population_tags_data/sample_first.xlsx: 0.25s
Processed file ../tmp/population_tags_data/sample_first.xlsx: 0.25s


In [61]:
full_df["research_scale"].values[0] = ["Small farms", "Medium/Large Scale Farms", 
                                          "Value Chains", "Markets", "Enabling Environment",
                                          "Food Systems", "Others"]

In [63]:
excel_writer.ExcelWriter().save_df_in_excel(full_df, "../tmp/population_tags_data/sample_first.xlsx")

Saving...
Saved to ../tmp/population_tags_data/sample_first.xlsx


In [64]:
elastic.ElasticClient("18.214.78.73:9200", "18.214.78.73:5601",
                      os.getenv("ELASTIC_USER"), os.getenv("ELASTIC_PASSWORD")).update_docs(
    full_df,
    properties={
        'title':      {'type': 'text'},
        'abstract': {'type': 'text'},
        "edit_url": {"type": "keyword"},
        "comments": {"type": "text"},
        "tags": {"type": "keyword"},
        "responsible_person": {"type": "keyword"},
        "labelled": {"type": "keyword"},
        "population tags": {"type": "keyword"},
        "sub_populations": {"type": "keyword"},
        "sub_populations_details": {'type': 'text'},
        "research_scale": {"type": "keyword"},
        "population_details": {"type": "text"},
        "interventions_extracted": {"type": "text"}
    },
    index_name='population_tags',
    time_field=None,
    create_search=False,
    drop_index=True,
    field_for_url_with_template="edit_url",
    fields_for_url=[],
    webservice_edit_host="18.214.78.73:8501"
)

Creating 'population_tags' Elasticsearch index
Preparing bulk operation
Indexing 1000 docs...
Creating 'population_tags' Kibana index pattern object
Done


# Prepare data for study design type

In [96]:
df = excel_reader.ExcelReader().read_df_from_excel("../tmp/cosai_data/subfolder-1.xlsx")

Read file ../tmp/cosai_data/subfolder-1.xlsx: 3.75s
Processed file ../tmp/cosai_data/subfolder-1.xlsx: 2.43s


In [66]:
from text_processing import search_engine_insensitive_to_spelling

folder_to_save = "../model/search_index_study_design_df"
small_search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
    load_abbreviations = True, columns_to_process=["title", "abstract"])
if os.path.exists(folder_to_save):
    small_search_engine_inverted_index.load_model(folder_to_save)
else:
    small_search_engine_inverted_index.create_inverted_index(df)
    small_search_engine_inverted_index.save_model(folder_to_save)

In [67]:
from interventions_labeling_lib import interventions_search_for_labeling
interventions_search_for_labeling = reload(interventions_search_for_labeling)
from text_processing import population_tags_finder
population_tags_finder = reload(population_tags_finder)
from text_processing import all_column_filler
all_column_filler = reload(all_column_filler)

_all_column_filler = all_column_filler.AllColumnFiller()
df = _all_column_filler.fill_columns_for_df(df, small_search_engine_inverted_index, _abbreviations_resolver,
                                                settings_json = {"columns":[
   {"column_filler_class":"ColumnFiller", "column_name": "study_type_keywords",
    "column_dictionary":"../data/study_type_keywords.xlsx"},]})

Started processing  {'column_filler_class': 'ColumnFiller', 'column_name': 'study_type_keywords', 'column_dictionary': '../data/study_type_keywords.xlsx'}
High level label: RCT
High level label: quasi-RCT
High level label: quasi experimental
High level label: quasi experiment
High level label: causal
High level label: comparative
High level label: descriptive
High level label: correlational
High level label: experimental research
High level label: field study
High level label: modelling
High level label: review
High level label: interview
High level label: respondents
High level label: laboratory
Labelled articles with outcomes: 3485
Processed for 2.4030704498291016s


In [68]:
df["Taken"] = 0

In [69]:
df["labels"] = ""
for i in range(len(df)):
    df["labels"].values[i] = []

In [70]:
excel_writer.ExcelWriter().save_df_in_excel(df, "all_sentences_study_design.xlsx")

Saving...
Saved to all_sentences_study_design.xlsx


In [71]:
found_words = {}
for i in range(len(df)):
    if df["Taken"].values[i] == 1:
        continue
    for w in df["study_type_keywords"].values[i]:
        if w not in found_words:
            found_words[w] = []
        found_words[w].append(i)

In [72]:
for w in found_words:
    print(w, len(found_words[w]))

review 1368
modelling 472
field study 172
interview 317
respondents 237
laboratory 586
comparative 386
descriptive 66
causal 79
RCT 11
quasi experimental 4
experimental research 12
correlational 1


In [73]:
chosen_ids = set()

In [74]:
chosen_ids.update(found_words["correlational"])
chosen_ids.update(found_words["experimental research"])
chosen_ids.update(found_words["quasi experimental"])
chosen_ids.update(found_words["RCT"])
len(chosen_ids)

28

In [75]:
for word in ["review", "modelling", "field study", "interview", "respondents", "laboratory", "comparative", "descriptive", "causal"]:
    np.random.shuffle(found_words[word])
    chosen_ids.update(found_words[word][:20])
len(chosen_ids)

205

In [76]:
wo_tags = []
for i in range(len(df)):
    if not df["study_type_keywords"].values[i]:
        wo_tags.append(i)
np.random.shuffle(wo_tags)

In [77]:
chosen_ids.update(wo_tags[:45])
len(chosen_ids)

250

In [78]:
for i in chosen_ids:
    df["Taken"].values[i] = 1

In [79]:
excel_writer.ExcelWriter().save_df_in_excel(df, "all_sentences_study_design.xlsx")

Saving...
Saved to all_sentences_study_design.xlsx


In [83]:
excel_writer.ExcelWriter().save_df_in_excel(utils.add_responsible_persons(df.take(list(chosen_ids)).copy(),persons=["Jaron"]), "../tmp/study_design_type_data/sample_first.xlsx")

Saving...
Saved to ../tmp/study_design_type_data/sample_first.xlsx


In [101]:
df = excel_reader.ExcelReader().read_df_from_excel("../tmp/study_design_type_data/sample_first.xlsx")

Read file ../tmp/study_design_type_data/sample_first.xlsx: 0.06s
Processed file ../tmp/study_design_type_data/sample_first.xlsx: 0.03s


In [103]:
elastic.ElasticClient("18.214.78.73:9200", "18.214.78.73:5601",
                      os.getenv("ELASTIC_USER"), os.getenv("ELASTIC_PASSWORD")).update_docs(
    df,
    properties={
        'title':      {'type': 'text'},
        'abstract': {'type': 'text'},
        "edit_url": {"type": "keyword"},
        "comments": {"type": "text"},
        "tags": {"type": "keyword"},
        "responsible_person": {"type": "keyword"},
        "labelled": {"type": "keyword"},
        "labels": {"type": "keyword"}
    },
    index_name='study_design_type',
    time_field=None,
    create_search=False,
    drop_index=True,
    field_for_url_with_template="edit_url",
    fields_for_url=[],
    webservice_edit_host="18.214.78.73:8501"
)

Creating 'study_design_type' Elasticsearch index
Preparing bulk operation
Indexing 250 docs...
Creating 'study_design_type' Kibana index pattern object
Done


In [17]:
df = excel_reader.ExcelReader().read_df_from_excel("labelled_multi_modelling_JP.xlsx")

Read file labelled_multi_modelling_JP.xlsx: 0.13s
Processed file labelled_multi_modelling_JP.xlsx: 0.06s


In [21]:
cnt_all, cnt_not_found = 0, 0
for i in range(len(df)):
    if df["Interviews"].values[i] == 1:
        cnt_all += 1
        if "interview" not in df["title"].values[i].lower() and "interview" not in df["abstract"].values[i].lower():
            print(i)
            print(df["title"].values[i])
            print("%%%")
            print(df["abstract"].values[i])
            print("%%%")
            cnt_not_found += 1
cnt_all, cnt_not_found

2
Thesis (MEd)
%%%
The purpose of the study was to investigate determinants of teachers' classroom . effectiveness on pupils' performance in KCPE in public primary schools within Maseno division, Kisumu west district. Kenya. The study was guided by the following research questions; how does teacher characteristic such as (Age, Gender, Academic Qualifications, Professional qualifications, and teaching experience) influence teachers' classroom effectiveness on pupils' performance in KCPE?; To what extent does class size influence teachers' effectiveness on pupils' performance in KCPE?, To what extent do teachers' in-service courses influence teachers' effectiveness on pupils' performance in KCPE?, How does teaching and learning activities and resources influence teachers' effectiveness on pupils' performance in KCPE? This study was based on Production function theory in education. The study adopted a descriptive survey design since it aimed at assessing the current situation and ongoing 

(34, 9)

In [7]:
from text_processing import search_engine_insensitive_to_spelling
from text_processing import all_column_filler
df = excel_reader.ExcelReader().read_df_from_excel("../tmp/study_type_data/train_keywords_plus_multi.xlsx")
search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
        load_abbreviations = True)
search_engine_inverted_index.create_inverted_index(df)

Read file ../tmp/study_type_data/train_keywords_plus_multi.xlsx: 1.33s
Processed file ../tmp/study_type_data/train_keywords_plus_multi.xlsx: 0.85s
Processed 0 articles
Processed 14541 articles
Processed 0 abbreviations
Processed 3000 abbreviations
Processed 6000 abbreviations
Processed 9000 abbreviations
Processed 12000 abbreviations
Processed 15000 abbreviations
Processed 18000 abbreviations
Processed 21000 abbreviations
Processed 24000 abbreviations
Processed 27000 abbreviations
Processed 27129 abbreviations


In [8]:
_all_column_filler = all_column_filler.AllColumnFiller()
df = _all_column_filler.fill_columns_for_df(
    df, search_engine_inverted_index, _abbreviations_resolver, settings_json = {"columns":[
        {"column_filler_class":"ColumnFiller", "column_name": "study_type_keywords", 
         "keep_hierarchy": False, "column_dictionary":"../data/study_design_types_keywords_to_search.xlsx",
        "column_details": "study_type_keywords_details"},
    ]})

Started processing  {'column_filler_class': 'ColumnFiller', 'column_name': 'study_type_keywords', 'keep_hierarchy': False, 'column_dictionary': '../data/study_design_types_keywords_to_search.xlsx', 'column_details': 'study_type_keywords_details'}
High level label: Book chapter
High level label: Field study
High level label: Greenhouse study
High level label: Impact evaluation
High level label: Laboratory study
High level label: Meta analysis
High level label: Modelling study
High level label: Observational study
High level label: Randomized controlled trials
High level label: Review paper
High level label: Systematic review
High level label: Interview
High level label: Survey/Questionnaire
High level label: Experimental research
High level label: Economic simulation
High level label: Landscape/environmental simulation
High level label: Other simulation
High level label: Policy analysis
Labelled articles with outcomes: 11467
Processed for 7.460175037384033s


In [9]:
df = df[["title", "abstract", "study_type_keywords", "study_type_keywords_details"]].copy()
environmental_keywords = ["environmental", "meteorological"," landscape", "land use", "environment", "hydrological", "agroecological",
                         "ecological"]
economics_keywords = ["economic", "econometric", "economical"]
for label in ["Observational study", "Modelling study", "Laboratory study", "Review paper", "Field study",
                "Economic simulation", "Landscape/environmental simulation"]:
    df[label] = 0
for i in range(len(df)):
    for label in df["study_type_keywords"].values[i]:
        if label in df.columns:
            df[label].values[i] = 1
    for keywords, label in [(environmental_keywords, "Landscape/environmental simulation"), (economics_keywords, "Economic simulation")]:
        if "Modelling study" in df["study_type_keywords"].values[i]:
            for key_word in keywords:
                if key_word in df["title"].values[i].lower() or key_word in df["abstract"].values[i].lower():
                    df[label].values[i] = 1
                    break

In [10]:
excel_writer.ExcelWriter().save_df_in_excel(df, "train_keywords_plus_multi.xlsx")

Saving...
Saved to train_keywords_plus_multi.xlsx


# Prepare author affiliations mapping

In [ ]:
countries = {}
for file in os.listdir("../tmp/cosai_data"):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/cosai_data", file))
    for i in range(len(df)):
        country_name = ""
        if len(df["affiliation"].values[i].split(",")) > 1:
            country_name = df["affiliation"].values[i].split(",")[-1].strip()
            country_name = re.sub("\d+", "", country_name).strip()
            country_name = remove_articles(country_name)
        if not country_name.strip():
            continue
        if country_name not in countries:
            countries[country_name] = 0
        countries[country_name] += 1
excel_writer.ExcelWriter().save_data_in_excel(countries.items(), ['country', "count"], "countries_count.xlsx")

In [68]:
excel_writer.ExcelWriter().save_data_in_excel(mappings.items(), ["raw_affiliation", "processed_affiliation"], "affiliations_to_label.xlsx")

Saving...
Saved to affiliations_to_label.xlsx


In [80]:
df_mapped["final_affiliation"] = ""
for i in range(len(df_mapped)):
    if len(df_mapped["ner_affiliation"].values[i].split()) == 1 and len(df_mapped["processed_affiliation"].values[i].split()) > 2:
        df_mapped["final_affiliation"].values[i] = df_mapped["processed_affiliation"].values[i]
    else:
        country_name = ""
        affiliation = ",".join(df_mapped["processed_affiliation"].values[i].split(",")[:-1])
        if len(df_mapped["processed_affiliation"].values[i].split(",")) > 1:
            country_name = df_mapped["processed_affiliation"].values[i].split(",")[-1]
            country_name = re.sub("\d+", "", country_name).strip()
            if country_name.strip() in mapping_countries:
                country_name = mapping_countries[country_name.strip()]
            country_name = to_camel_case(country_name)
        
        all_parts = []
        for aff in df_mapped["ner_affiliation"].values[i].split(";"):
            if aff.strip():
                all_parts.append((aff, get_rank(aff, default_rank=100)))
        if affiliation.strip():
            all_parts.append((affiliation, get_rank(affiliation, default_rank=101)))
        all_parts = sorted(all_parts, key=lambda x: (x[1], len(x[0])))
        if all_parts:
            aff = remove_brackets(all_parts[0][0])
            df_mapped["final_affiliation"].values[i] = to_camel_case(
                aff if not country_name else (aff + ", " + country_name))
    df_mapped["final_affiliation"].values[i] = remove_articles(df_mapped["final_affiliation"].values[i])
    if re.search("\d+", df_mapped["final_affiliation"].values[i]):
        df_mapped["final_affiliation"].values[i] = ""

In [81]:
excel_writer.ExcelWriter().save_df_in_excel(df_mapped, "affiliations_sample.xlsx")

Saving...
Saved to affiliations_sample.xlsx


In [86]:
train_data_df = excel_reader.ExcelReader().read_df_from_excel("train_affiliation.xlsx")
train_data_df_set = set(train_data_df["Sentence"].values)

Read file train_affiliation.xlsx: 0.01s
Processed file train_affiliation.xlsx: 0.00s


In [88]:
df_mapped.columns

Index(['raw_affiliation', 'processed_affiliation', 'ner_affiliation',
       'final_affiliation'],
      dtype='object')

In [101]:
idx_with_processed = []
idx_with_processed_usa = []
idx_wo_processed = []
for i in range(len(df_mapped)):
    if df_mapped["raw_affiliation"].values[i].strip() in train_data_df_set:
        continue
    if "united states" in df_mapped["final_affiliation"].values[i].lower() or "usa" in df_mapped["final_affiliation"].values[i].lower():
        idx_with_processed_usa.append(i)
    elif df_mapped["final_affiliation"].values[i].strip():
        idx_with_processed.append(i)
    else:
        idx_wo_processed.append(i)

In [102]:
np.random.shuffle(idx_with_processed)
np.random.shuffle(idx_with_processed_usa)
np.random.shuffle(idx_wo_processed)

In [104]:
excel_writer.ExcelWriter().save_df_in_excel(df_mapped.take(idx_with_processed_usa[:150]+idx_with_processed[:320]+idx_wo_processed[:30]), "sample_affiliations_Tom.xlsx")

Saving...
Saved to sample_affiliations_Tom.xlsx


In [82]:
from collections import Counter
excel_writer.ExcelWriter().save_data_in_excel(Counter(df_mapped["final_affiliation"].values).items(), ['univ', "count"], "univ_count.xlsx")

Saving...
Saved to univ_count.xlsx


In [1]:
import elasticsearch
import elasticsearch.helpers
import os

es = elasticsearch.Elasticsearch(["http://%s:%s@%s/"%(
    os.getenv("ELASTIC_USER"), os.getenv("ELASTIC_PASSWORD"), "18.214.78.73:9200")],
    timeout=30, max_retries=5, retry_on_timeout=True)

In [5]:
df_population_tags = excel_reader.ExcelReader().read_df_from_excel("../tmp/population_tags_data/sample_first.xlsx")

Read file ../tmp/population_tags_data/sample_first.xlsx: 0.84s
Processed file ../tmp/population_tags_data/sample_first.xlsx: 0.52s


In [7]:
df_population_tags["population_tags_labelled"] = ""
df_population_tags["sub_populations_labelled"] = ""
df_population_tags["sub_populations_details_labelled"] = ""
df_population_tags["population_details_labelled"] = ""
df_population_tags["interventions_extracted_labelled"] = ""
df_population_tags["research_scale_labelled"] = ""
df_population_tags["tags_labelled"] = ""

In [8]:
for r in elasticsearch.helpers.scan(es, query={}, index="population_tags"):
    idx = int(r["_id"])
    df_population_tags["population_tags_labelled"].values[idx] = r["_source"]["population tags"]
    df_population_tags["sub_populations_labelled"].values[idx] = r["_source"]["sub_populations"]
    df_population_tags["sub_populations_details_labelled"].values[idx] = r["_source"]["sub_populations_details"]
    df_population_tags["population_details_labelled"].values[idx] = r["_source"]["population_details"]
    df_population_tags["interventions_extracted_labelled"].values[idx] = r["_source"]["interventions_extracted"]
    df_population_tags["research_scale_labelled"].values[idx] = r["_source"]["research_scale"]
    df_population_tags["tags_labelled"].values[idx] = r["_source"]["tags"]

In [9]:
excel_writer.ExcelWriter().save_df_in_excel(df_population_tags, "df_population_tags.xlsx")

Saving...
Saved to df_population_tags.xlsx


In [36]:
df_study_design_type = excel_reader.ExcelReader().read_df_from_excel("../tmp/study_design_type_data/sample_first.xlsx")

Read file ../tmp/study_design_type_data/sample_first.xlsx: 0.09s
Processed file ../tmp/study_design_type_data/sample_first.xlsx: 0.05s


In [38]:
for r in elasticsearch.helpers.scan(es, query={}, index="study_design_type"):
    idx = int(r["_id"])
    df_study_design_type["tags"].values[idx] = r["_source"]["tags"]
    df_study_design_type["labels"].values[idx] = r["_source"]["labels"]
    df_study_design_type["comments"].values[idx] = r["_source"]["comments"]

In [46]:
new_df = df_study_design_type[["title", "abstract", "id"]].copy()

In [41]:
all_labels = set()
for i in range(len(df_study_design_type)):
    all_labels.update(df_study_design_type["labels"].values[i])

In [47]:
for label in all_labels:
    new_df[label] = 0

In [48]:
for i in range(len(df_study_design_type)):
    for label in df_study_design_type["labels"].values[i]:
        new_df[label].values[i] = 1

In [51]:
excel_writer.ExcelWriter().save_df_in_excel(new_df, "df_study_design_type_JP.xlsx")

Saving...
Saved to df_study_design_type_JP.xlsx


In [70]:
df_study_types = excel_reader.ExcelReader().read_df_from_excel("../tmp/labelled_study_types.xlsx")

Read file ../tmp/labelled_study_types.xlsx: 2.51s
Processed file ../tmp/labelled_study_types.xlsx: 1.33s


In [72]:
ind_to_take_economic = set()
for i in range(len(df_study_types)):
    if df_study_types["Economic simulation"].values[i] > 0.1 and df_study_types["Economic simulation"].values[i] < 0.65:
        ind_to_take_economic.add(i)
len(ind_to_take_economic)

34

In [73]:
ind_to_take_landscape = set()
for i in range(len(df_study_types)):
    if df_study_types["Landscape/environmental simulation"].values[i] > 0.02 and df_study_types["Landscape/environmental simulation"].values[i] < 0.7:
        ind_to_take_landscape.add(i)
len(ind_to_take_landscape)

50

In [75]:
ind_to_take_other = set()
for i in range(len(df_study_types)):
    if df_study_types["Other simulation"].values[i] > 0.3 and df_study_types["Other simulation"].values[i] < 0.65:
        ind_to_take_other.add(i)
len(ind_to_take_other)

70

In [77]:
all_inds = ind_to_take_other.union(ind_to_take_economic).union(ind_to_take_landscape)
len(all_inds)

150

In [81]:
excel_writer.ExcelWriter().save_df_in_excel(df_study_types.take(list(sorted(list(all_inds)))), "sample_modelling_more.xlsx",
                                           column_probabilities=['Observational study', 'Modelling study',
       'Laboratory study', 'Review paper', 'Field study',
       'Economic simulation', 'Landscape/environmental simulation',
       'Other simulation'])

Saving...
Saved to sample_modelling_more.xlsx


## NER pesticides experiments

In [ ]:
def filter_phrase(narrow_concept, threshold, less_strict):
    frequent_words = 0
    add_to_dict = True
    narrow_concept = re.sub(r"\b(\d+th|\d+rd|\d+st|\d+nd)\b", " ", narrow_concept).strip()
    narrow_concept = re.sub("[\[\],!?\{\}]", "", narrow_concept).strip()
    if len(narrow_concept) < 2 or len(re.sub("\d", "", narrow_concept).strip()) < 2:
        return True
    for word in narrow_concept.split():
        if word in text_normalizer.freq_words or text_normalizer.get_rank_of_word(word) < 0.99 or word in text_normalizer.stopwords_all:
            frequent_words += 1
    if len(narrow_concept.split()) == frequent_words:
        if frequent_words > 1:
            score = text_normalizer.calculate_common_topic_score(narrow_concept)
            if score <= threshold:
                add_to_dict = False
        else:
            add_to_dict = less_strict and narrow_concept != "" and text_normalizer.calculate_common_topic_score(narrow_concept) >= threshold
    return not add_to_dict

pesticides_dictionary = {}

for file in os.listdir("../tmp/processed_deduplicated_cosai_processed"):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/processed_deduplicated_cosai_processed", file))
    for i in range(len(df)):
        for word in df["pesticides_found"].values[i]:
            word_lemmatized = ""
            if len(word.split()) == 1:
                word_lemmatized = text_normalizer.lemmatize_word(word)
            else:
                word_lemmatized = " ".join(word.split()[:-1]) + " " + text_normalizer.lemmatize_word(word.split()[-1])
            if word_lemmatized != word:
                pesticides_dictionary[word_lemmatized] = word

all_found_pesticides = set()
for file in os.listdir("../tmp/processed_deduplicated_cosai_processed"):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/processed_deduplicated_cosai_processed", file))
    df["pesticides_found_all"] = ""
    df["cabi_codes"] = ""
    for i in range(len(df)):
        df["pesticides_found_all"].values[i] = df["pesticides_found"].values[i].copy()
        if len(df["pesticides_found"].values[i]):
            for w in df["ner_pesticides"].values[i]:
                should_be_filtered = False
                for word in ["fao", "usa", "ukraine", "baseline", "diameter", "denmark"]:
                    if re.search(r"\b%s\b" % word, w.lower()):
                        should_be_filtered = True
                        break
                for word in ["t.", "e.g."]:
                    if w.lower() == word:
                        should_be_filtered = True
                        break
                if should_be_filtered:
                    continue
                if re.search("^\w+-$", w) and re.search("^\w+-$", w).group(0) == w:
                    continue
                if not filter_phrase(w.lower(), 0.8, False):
                    word_to_use = " ".join([
                        (word_part if text_normalizer.is_abbreviation(word_part) else word_part.lower()) for word_part in w.split() if word_part.lower() not in text_normalizer.stopwords_all])
                    if word_to_use in pesticides_dictionary:
                        word_to_use = pesticides_dictionary[word_to_use]
                    word_to_use = re.sub("^[-,!;,\.\?]", "", word_to_use.strip())
                    if ")" not in word_to_use:
                        word_to_use = re.sub(r"\(\w+\b", "", word_to_use)
                    if "(" not in word_to_use:
                        word_to_use = word_to_use.replace(")", "")
                    if not word_to_use.strip():
                        continue
                    df["pesticides_found_all"].values[i].append(word_to_use)
                    all_found_pesticides.add(word_to_use)
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join("../tmp/processed_deduplicated_cosai_processed", file))

excel_writer.ExcelWriter().save_df_in_excel(pd.DataFrame({
    "pesticides_extracted": list(all_found_pesticides)
}), "pesticides_extracted.xlsx")

# Processing data

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [7]:
from text_processing import search_engine_insensitive_to_spelling
from text_processing import all_column_filler

folder_with_processed = "../tmp/processed_deduplicated_cosai_processed"
folder_with_files = "../tmp/processed_deduplicated_cosai"
folder_with_search_indices = "../model/cosai_index_deduplicated/"
os.makedirs(folder_with_processed, exist_ok=True)
for file in os.listdir(folder_with_files):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_files, file))
    search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
        load_abbreviations = True)
    file_part = file.split(".")[0]
    if os.path.exists(folder_with_search_indices + file_part):
        search_engine_inverted_index.load_model(folder_with_search_indices + file_part)
    else:
        search_engine_inverted_index.create_inverted_index(df)
        search_engine_inverted_index.save_model(folder_with_search_indices + file_part)
    
    _all_column_filler = all_column_filler.AllColumnFiller()
    df["identificators"] = ""
    df["raw_authors"] = df["authors"]
    df["raw_affiliation"] = df["affiliation"]
    df["raw_corporate_author"] = df["corporate_author"]
    for i in range(len(df)):
        df["authors_processed"].values[i] = [author.strip() for author in df["authors"].values[i].split(";") if author.strip()]
    
    df = _all_column_filler.fill_columns_for_df(
        df, search_engine_inverted_index, _abbreviations_resolver, settings_json = {"columns":[
            {"column_filler_class":"KeywordNormalizer"},
            {"column_filler_class":"GeoNameFinder", "geo_regions_file": "../data/GeoRegionsDifferentMapping.xlsx"},
            {"column_filler_class":"CropsSearch", "file_dictionary":"../data/map_plant_products.xlsx", 
             "column_name":"plant_products_search"},
            {"column_filler_class":"CropsSearch", "file_dictionary":"../data/map_animal_products.xlsx",
             "column_name":"animal_products_search"},
            {"column_filler_class":"CropsSearch", "column_name": "animals_found", "keep_hierarchy": False,
             "file_dictionary":"../data/map_animals.xlsx"},
            {"column_filler_class":"CropsSearch", "column_name": "plant_diseases_found", "keep_hierarchy": False,
             "file_dictionary":"../data/map_plant_diseases.xlsx"},
            {"column_filler_class":"CropsSearch", "column_name": "animal_diseases_found", "keep_hierarchy": False,
             "file_dictionary":"../data/map_animal_diseases.xlsx"},
            {"column_filler_class":"CropsSearch", "column_name": "pests_found", "keep_hierarchy": False,
             "file_dictionary":"../data/map_pests.xlsx"},
            {"column_filler_class":"CropsSearch", "column_name": "pesticides_found", "keep_hierarchy": False,
             "file_dictionary":"../data/map_pesticides.xlsx"},
            {"column_filler_class":"PopulationTagsFinder", "column_name": "farm_size_tags"},
            {"column_filler_class":"ColumnFiller", "column_name": "ecological_zones", 
             "keep_hierarchy": False, "column_dictionary":"../data/map_agroecological_zones.xlsx"},
            {"column_filler_class":"InterventionsSearchForLabeling", "file":"../tmp/interventions.xlsx"},
            {"column_filler_class": "StudyTypeLabeller", "folder": "../model/study_type_multi",
             "meta_folder": "../model/study_type_multi_meta_agg",
             "scibert_model_folder": "../model/scibert_scivocab_uncased"},
            {"column_filler_class": "OutcomesFinder",
             "model_folder": "../model/bert_exp_outcome_sentences_new_multilabel_15epoch_1300_mixed_0.7"},
            {"column_filler_class":"PopulationTagsFinder", "column_name": "farm_size_tags"},
            {"column_filler_class":"ColumnFiller", "column_name": "ecological_zones", 
             "keep_hierarchy": False, "column_dictionary":"../data/map_agroecological_zones.xlsx"},
            {"column_filler_class":"ColumnFiller", "column_name": "sex_mentioned", 
             "keep_hierarchy": False, "column_dictionary":"../data/map_sex.xlsx"},
            {"column_filler_class":"ColumnFiller", "column_name": "age_mentioned", 
             "keep_hierarchy": False, "column_dictionary":"../data/map_age.xlsx"},
            {"column_filler_class":"ColumnFiller", "column_name": "equity_mentioned", 
             "keep_hierarchy": False, "column_dictionary":"../data/map_equity.xlsx"},
            {"column_filler_class":"ColumnFiller", "column_name": "agricultural_workers_mentioned", 
             "keep_hierarchy": False, "column_dictionary":"../data/map_agricultural_workers.xlsx"}
        ]})
    for column in ["sex_mentioned", "age_mentioned", "equity_mentioned",
                   "agricultural_workers_mentioned"]:
        for i in range(len(df)):
            if not len(df[column].values[i]):
                df[column].values[i] = ["Unspecified"]
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_with_processed, file_part + ".xlsx"))

Read file /hdd2/data/maryia_pavlovets/cosai_data/subfolder-0.xlsx: 3.57s
Processed file /hdd2/data/maryia_pavlovets/cosai_data/subfolder-0.xlsx: 1.76s
Started processing  {'column_filler_class': 'KeywordNormalizer'}
Processed for 12.144060850143433s
Started processing  {'column_filler_class': 'GeoNameFinder'}
Read file ../data/provinces.xlsx: 2.50s
Processed file ../data/provinces.xlsx: 0.98s
Read file ../data/districts.xlsx: 1.21s
Processed file ../data/districts.xlsx: 1.04s
Processed 0 items
Processed 5000 items
Processed 10000 items
Processed 15000 items
Processed 20000 items
Processed 25000 items
Processed 29999 items
Processed for 1740.6950368881226s
Started processing  {'column_filler_class': 'CropsSearch', 'file_dictionary': '../data/map_plant_products.xlsx', 'column_name': 'plant_products_search'}
Read file ../data/map_plant_products.xlsx: 0.06s
Processed file ../data/map_plant_products.xlsx: 0.02s
Found crops: plant_products_search
Number of articles that have plant_products_s

I0822 20:01:00.473383 140070156113728 estimator.py:201] Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/study_type_multi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f625cb15470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
INFO:tensorflow:Writing example 0 of 30000


I0822 20:01:02.114542 140070156113728 base_bert_model.py:478] Writing example 0 of 30000


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Running local_init_op.


I0823 00:28:19.445317 140070156113728 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0823 00:28:20.221035 140070156113728 session_manager.py:493] Done running local_init_op.


Processed for 1157.3244895935059s
Started processing  {'column_filler_class': 'NERExtractor', 'column_name': 'ner_pesticides', 'columns_to_process': ['title', 'abstract'], 'model_folder': '/hdd2/data/maryia_pavlovets/pecticide_ext_model_180-60_iter/'}
Processed 0 rows
Processed 3000 rows
Processed 6000 rows
Processed 9000 rows
Processed 12000 rows
Processed 15000 rows
Processed 18000 rows
Processed 21000 rows
Processed 24000 rows
Processed 27000 rows
Processed for 455.9932096004486s
Saving...
Saved to /hdd2/data/maryia_pavlovets/processed_cosai/subfolder-2.xlsx


# Affilaition mapping

In [26]:
from text_processing import affiliations_extractor
_affiliations_extractor = affiliations_extractor.AffiliationsExtractor()
_affiliations_extractor.extract_affiliations_and_remap("../tmp/processed_deduplicated_cosai_processed",
                                                       "../tmp/processed_deduplicated_cosai_processed",
                                                       "../tmp/affiliations_mapping",
                                                       "../tmp/affilliations_counts.xlsx")

In [27]:
_affiliations_extractor.map_affilations_from_clustered_affiliations("../tmp/processed_deduplicated_cosai_processed",
                                                                    "../tmp/processed_deduplicated_cosai_processed",
                                                                    "../tmp/affiliations_mapping_fixed",
                                                                    column_affiliation_mappings={"column_name_to_map":"processed_affiliation",
                                                                                                 "processed_column_name": "affiliation_processed"},
                                                                    column_corporate_author_mappings={"column_name_to_map":"processed_corporate_author",
                                                                                                 "processed_column_name": "corporate_author_processed"})

## Interventions finding

In [56]:
all_found_interventions = set()
all_df = pd.DataFrame()
for file in ["../tmp/interventions.xlsx", "../tmp/cosai_interventions.xlsx"]:
    df = excel_reader.ExcelReader().read_df_from_excel(file)
    indices_to_take = []
    for i in range(len(df)):
        vals = tuple(df.values[i])
        if vals not in all_found_interventions:
            all_found_interventions.add(vals)
            indices_to_take.append(i)
    all_df = pd.concat([all_df, df], axis=0)
print("All interventions cnt: ", len(all_df))
excel_writer.ExcelWriter().save_df_in_excel(all_df, "all_interventions_with_cosai.xlsx")

Read file ../tmp/interventions.xlsx: 3.50s
Processed file ../tmp/interventions.xlsx: 1.48s
Read file ../tmp/cosai_interventions.xlsx: 1.52s
Processed file ../tmp/cosai_interventions.xlsx: 1.42s
All interventions cnt:  58040
Saving...
Saved to all_interventions_with_cosai.xlsx


In [ ]:
from text_processing import search_engine_insensitive_to_spelling
from text_processing import all_column_filler

for file in os.listdir(folder_with_processed):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_processed, file))
    search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
        load_abbreviations = True)
    file_part = file.split(".")[0]
    if os.path.exists(folder_with_search_indices + file_part):
        search_engine_inverted_index.load_model(folder_with_search_indices + file_part)
    else:
        search_engine_inverted_index.create_inverted_index(df)
        search_engine_inverted_index.save_model(folder_with_search_indices + file_part)
    
    _all_column_filler = all_column_filler.AllColumnFiller()
    df = _all_column_filler.fill_columns_for_df(
        df, search_engine_inverted_index, _abbreviations_resolver, settings_json = {"columns":[
            {"column_filler_class":"InterventionsSearchForLabeling", "file":"../tmp/all_interventions_with_cosai.xlsx"},
        ]})
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_with_processed, file_part + ".xlsx"))

Read file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 28.05s
Processed file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 92.74s
Started processing  {'column_filler_class': 'InterventionsSearchForLabeling', 'file': '../tmp/all_interventions_with_cosai.xlsx'}
Read file ../tmp/all_interventions_with_cosai.xlsx: 3.25s
Processed file ../tmp/all_interventions_with_cosai.xlsx: 3.21s


In [109]:
from synonyms_module import synonyms_processor

_synonyms_all = synonyms_processor.SynonymsProcessor("../model/synonyms_retrained_new/")

def get_embed_for_phrase(text):
    for w in ['programme',
             'policy',
             'action',
             'program',
             'intervention aimed',
             'strategy',
             'initiative',
             'effort',
             'measure',
             'project',
             'outcome',
             'implementation',
             'policy instrument',
             'training program',
             'public policy',
             'counseling',
             'reform']:
        text = text.replace(w, " ")
    return _synonyms_all.get_average_embedding(text)

In [110]:
folder_with_processed = "../tmp/processed_cosai_all_data"
all_interventions = {}
all_interventions_dict = {}
for file in os.listdir(folder_with_processed):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_processed, file))
    for i in range(len(df)):
        for interv in df["interventions_found"].values[i]:
            interv_split_words = interv.split("/")
            if len(interv_split_words) < 3 or not interv_split_words[2].strip():
                continue
            raw_intervention = "/".join(interv_split_words[2:])
            intervention_group = "/".join(interv_split_words[:2])
            if intervention_group not in all_interventions:
                all_interventions[intervention_group] = set()
            all_interventions[intervention_group].add(raw_intervention)
            if raw_intervention not in all_interventions_dict:
                all_interventions_dict[raw_intervention] = 0
            all_interventions_dict[raw_intervention] += 1

Read file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 30.26s
Processed file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 75.75s
Read file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 25.06s
Processed file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 58.60s
Read file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 22.15s
Processed file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 55.64s
Read file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 23.41s
Processed file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 57.12s
Read file ../tmp/processed_cosai_all_data\data_year_2004.xlsx: 26.61s
Processed file ../tmp/processed_cosai_all_data\data_year_2004.xlsx: 60.65s
Read file ../tmp/processed_cosai_all_data\data_year_2005.xlsx: 31.76s
Processed file ../tmp/processed_cosai_all_data\data_year_2005.xlsx: 70.04s
Read file ../tmp/processed_cosai_all_data\data_year_2006.xlsx: 29.86s
Processed file ../tmp/processed_cosai_all_data\data_year_200

In [111]:
from scipy import spatial
from sklearn.cluster import KMeans
import hdbscan

remappings = {}
word2cluster_interv = {}
print_res = True
for group in all_interventions:
    remappings[group] = {}
    texts = list(all_interventions[group])
    print(group, len(texts))
    word_embeds = []
    for i in range(len(texts)):
        word_embeds.append(get_embed_for_phrase(text_normalizer.normalize_text(texts[i])))
    if len(word_embeds) < 2:
        remappings[group][texts[0]] = texts[0]
        continue
    clusterer = hdbscan.HDBSCAN(min_samples=3, min_cluster_size=3)
    clusterer.fit(word_embeds)
    if print_res:
        print(clusterer.labels_.max(), clusterer.labels_)
    cluster_words = {}
    for idx in range(len(texts)):
        if clusterer.labels_[idx] not in cluster_words:
            cluster_words[clusterer.labels_[idx]] = []
        cluster_words[clusterer.labels_[idx]].append((texts[idx], word_embeds[idx]))
    for cluster_id in range(clusterer.labels_.max()+1):
        if cluster_id == -1:
            continue
        kmeans = KMeans(n_clusters=1, random_state=0).fit(
            [ word_embeds[idx] for idx in range(len(texts)) if  clusterer.labels_[idx] == cluster_id])
        all_words = []
        for word, embed in cluster_words[cluster_id]:
            all_words.append((
                word, all_interventions_dict[word], round(1 - spatial.distance.cosine(embed, kmeans.cluster_centers_[0]), 2)))
        all_words = sorted(all_words, key=lambda x: (x[1], x[2], -len(x[0].split())), reverse=True)
        if print_res:
            print("Cluster ", cluster_id, " Main word ", all_words[0][0])
            for w in all_words:
                print(w)
        for w in all_words:
            remappings[group][w[0]] = all_words[0][0]
        word2cluster_interv[all_words[0][0]] = [w[0].strip() for w in all_words]
    if -1 in cluster_words:
        for w, embed in cluster_words[-1]:
            remappings[group][w] = w

Technology intervention/crop 2118
98 [-1 -1 -1 ... -1 -1 84]
Cluster  0  Main word  seining practice
('seining practice', 1, 1.0)
('agroextractivist practice', 1, 1.0)
('articulatory practice', 1, 1.0)
Cluster  1  Main word  improving productivity
('improving productivity', 965, 0.96)
('promoting productivity', 26, 0.93)
('stabilizing productivity', 15, 0.92)
('improving productivity growth', 11, 0.92)
Cluster  2  Main word  silvopastoral system (SPS)
('silvopastoral system (SPS)', 1770, 0.97)
('spontaneous silvopastoral system (SPS)', 1, 0.92)
('silvopastoral system (SPS) boundary', 1, 0.9)
Cluster  3  Main word  weed detection technique
('weed detection technique', 4, 0.95)
('weed detection technology', 3, 0.95)
('time weed detection equipment', 1, 0.89)
Cluster  4  Main word  renin angiotensin aldosterone system
('renin angiotensin aldosterone system', 33, 1.0)
('autoguidance system', 2, 1.0)
('RYESTD system', 1, 1.0)
('sustenagro system', 1, 1.0)
('nicroelectromechanical system', 1

Cluster  44  Main word  earthing potato
('earthing potato', 2, 0.96)
('artificial defoliation potato', 1, 0.96)
('potato haulm remover', 1, 0.93)
Cluster  45  Main word  cultivation vegetable crop (VC)
('cultivation vegetable crop (VC)', 319, 0.95)
('growth vegetable crop (VC)', 49, 0.94)
('genetic improvement (GI) vegetable crop (VC)', 8, 0.91)
Cluster  46  Main word  rice production technology
('rice production technology', 173, 0.98)
('wheat production technology', 63, 0.96)
('cotton production technology', 53, 0.95)
('rice based production technology', 1, 0.94)
Cluster  47  Main word  witch weed
('witch weed', 47, 0.98)
('weed wiper', 4, 0.99)
('weed sod culture', 2, 0.99)
('slashing weed machete', 1, 0.96)
Cluster  48  Main word  fodder crop
('fodder crop', 11289, 0.91)
('cultivation fodder', 115, 0.96)
('fodder cultivation', 96, 0.96)
('cultivation fodder crop', 88, 0.98)
Cluster  49  Main word  deploying resistant barley cultivar
('deploying resistant barley cultivar', 1, 0.96)


Cluster  82  Main word  maize based cropping system (MBCS)
('maize based cropping system (MBCS)', 125, 0.94)
('cereal based cropping system (CS)', 92, 0.95)
('cassava based cropping system (CS)', 22, 0.94)
Cluster  83  Main word  cropping system (CS)
('cropping system (CS)', 17160, 0.99)
('alternate cropping system (CS)', 15, 0.93)
('adjustment cropping system (CS)', 9, 0.89)
('wild blueberry (WBB) cropping system (CS)', 3, 0.94)
('indigenous cropping system (CS)', 3, 0.88)
('specialized cropping system (CS)', 2, 0.9)
('adjusted cropping system (CS)', 2, 0.89)
('multi tier cropping system (CS)', 1, 0.99)
('spatial arrangement cropping system (CS)', 1, 0.95)
Cluster  84  Main word  winter cropping
('winter cropping', 74, 0.97)
('summer cropping', 53, 0.98)
('spring cropping', 33, 0.97)
Cluster  85  Main word  hybrid rice variety
('hybrid rice variety', 198, 0.89)
('drought tolerant (DT) crop', 163, 0.84)
('high yielding crop variety (HYV)', 35, 0.92)
('drought tolerant (DT) crop variety

30 [29 -1 -1 -1 14 21 24 -1 -1 -1 -1 -1 -1 -1 24 -1 -1 -1 -1 -1 -1 -1 29 -1
 -1 -1 -1 -1  8 -1 -1 -1 -1 16 26 -1 -1 -1 -1 17 -1 -1 -1 -1 11 29 -1 -1
 -1  4 10 14 28 -1 -1 -1 -1 -1 -1 -1 11 -1 -1 -1 -1 14  8 -1 -1 -1 -1 -1
 -1 -1 -1  1 -1 18 -1 -1 12 30 -1 -1 -1 -1 27 20 -1 -1 -1  3 10 -1 23 -1
 -1 -1 -1 21 -1 22 -1 -1 -1 21 -1 -1 -1 29  4 28 -1 -1 15 -1 29 -1 -1 -1
 -1 -1 -1 -1 -1 -1 23 -1 -1 24  2 15 -1 -1 -1 -1  7 -1 -1 11 10 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 10 -1 10 -1 -1 -1 -1 23 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1  9 -1 28 -1 -1 -1 -1 15 -1 -1 -1 -1 -1 -1 -1 22 -1
 16 -1  8 -1 -1  4 -1  5 -1 -1 -1 -1 -1 -1 -1 10 -1 -1 -1 -1 -1 -1 25 21
 -1 -1 23 11 -1 -1 -1 20 25 -1 -1 -1 -1 -1 -1 -1 21 -1 -1 -1 -1 -1 -1 28
 20 -1  3 -1 23 -1 -1 17 -1  0 -1 28 -1 -1 -1 -1 -1 19 -1 -1 19 -1 -1 14
  8 -1 -1 -1 -1 -1 -1 -1 -1 -1 15 11 20 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  8 -1 17 -1 16 -1  7 -1 -1  8  8 -1 -1 -1 -1 23 -1 18 -1 -1 -1 -1  8 -1
 -1 -1 18 -1 -1 -1 30 -1 -1 -1 -1 -1 -1 -1 -1 -1

C:\Users\Maryia_Ivanina\Anaconda3\envs\cornell\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


...

C:\Users\Maryia_Ivanina\Anaconda3\envs\cornell\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


16 [-1  5 15 -1 -1 -1 14  8 -1 14 -1 -1 -1 11 -1 -1  9 -1 -1 -1 14 15 15 -1
 -1 -1 15 -1 -1 -1  5 -1 -1 14 -1 -1 -1 -1 -1 -1 -1 14 10 -1 -1 -1 -1 -1
  7 -1 15 15 -1 -1 -1 15 15 15 -1 15 -1 -1 -1 -1 15 -1 -1 13 -1 -1 -1 -1
 15 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 15 -1 14 12 -1 -1 -1 14 -1 14 -1 -1 -1
 -1 -1 -1 14 -1 -1 16 15 15 -1 -1 -1 14 -1 -1 -1 -1 15 -1 -1 14  6 14 -1
 -1 -1 -1 -1 15 -1 -1 -1 -1 -1 14 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1  5 -1 -1 -1 14 -1 -1 -1 -1 -1 14 -1 -1 -1  0 -1  0 -1 -1 10 -1 -1 -1
 -1 15 14 -1 -1 -1 -1 -1 15  7 -1 -1 -1 -1 -1 14 -1  6 -1  3 11 -1 -1 -1
  1 -1 -1 15 -1 -1  7 -1 15 -1 -1 -1 -1  2 15 15 -1 -1 -1 -1 -1 -1 -1 -1
 15 -1 -1 -1 15 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 15 14 -1 -1 15 -1 15 -1
 14  1 15 -1 -1 -1 -1 -1 -1 -1 -1 15 -1 15 -1 -1 -1 -1 -1 -1  2 -1 -1 -1
 -1 -1 14 -1 -1 14 -1 14 -1  9 -1 -1 15 -1 -1  3 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 15 -1 -1 -1 -1 -1 -1  5 -1 -1 -1 -1 -1 15 -1 -1 -1 15 -1 -1 -1
 -1 -1 -1 -1 -1 -1 15 -1 -1 -1 -1 -1 -1  4 -1 -1

C:\Users\Maryia_Ivanina\Anaconda3\envs\cornell\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Cluster  16  Main word  forage harvester (FH)
('forage harvester (FH)', 339, 0.97)
('reel forage harvester (FH)', 1, 0.97)
('self propelled flail type flail forage harvester (FH)', 1, 0.95)
Mechanisation intervention/tractor 277
9 [-1 -1 -1  6  8  8 -1 -1 -1 -1  1  7 -1 -1 -1  8  6 -1 -1 -1  0 -1  6  8
 -1 -1 -1  6 -1  2 -1 -1  9 -1  8 -1  1 -1 -1 -1  6  8 -1  3 -1  6 -1  1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  4 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1  5 -1
  9  6  6 -1 -1 -1 -1 -1 -1 -1 -1  6 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  6
 -1 -1  6  6  1  6  1  6 -1 -1 -1 -1  8 -1  9 -1 -1  8 -1 -1 -1  8  6 -1
 -1  8 -1 -1 -1 -1 -1 -1  6 -1  9 -1 -1  8 -1 -1 -1  6 -1 -1  0  6  3 -1
 -1 -1 -1  6 -1 -1 -1  7 -1  0 -1 -1  8 -1 -1 -1 -1 -1  2  5 -1  6 -1  8
 -1  1 -1  8 -1  6 -1  6 -1  8 -1 -1 -1 -1  9 -1  8 -1  0 -1  6  6  8 -1
 -1 -1 -1 -1 -1  6 -1  8  6 -1  7  6 -1  6  8 -1  4 -1 -1  8 -1 -1 -1 -1
  7  6 -1  5  4 -1  2 -1  6 -1 -1  6 -1 -1  4 -1 -1 -1 -1 -1  6 -1  3 -1
 -1  6 -1 -1 -1 -1  8 -1  6  6  6 -1  8

Cluster  3  Main word  power transmission system
('power transmission system', 33, 0.91)
('remote control crawler type power machine', 2, 0.81)
('power quality (PQ) analyzer', 2, 0.73)
('power transmission system seed drill (SD)', 1, 0.89)
('power requirement low power consumption (PC) sensor node', 1, 0.83)
Cluster  4  Main word  inertia force
('inertia force', 29, 0.67)
('air compressor', 19, 0.75)
('axial force', 9, 0.72)
('torque meter', 8, 0.51)
('oleo dynamic torque amplifier', 1, 0.83)
Cluster  5  Main word  electric powertrain
('electric powertrain', 2, 0.98)
('electric bike', 1, 0.97)
('electric versus internal combustion (IC)', 1, 0.87)


In [112]:
for group in remappings:
    cnt = 0
    for w in remappings[group]:
        if w != remappings[group][w]:
            cnt += 1
    print(group, " remapped cnt: ", cnt)

Technology intervention/crop  remapped cnt:  531
Technology intervention/irrigation  remapped cnt:  150
Technology intervention/seed  remapped cnt:  188
Technology intervention/livestock  remapped cnt:  675
Technology intervention/other  remapped cnt:  2164
Ecosystem intervention/forest  remapped cnt:  449
Socioeconomic intervention/food  remapped cnt:  136
Socioeconomic intervention/community  remapped cnt:  99
Ecosystem intervention/conservation  remapped cnt:  95
Socioeconomic intervention/education  remapped cnt:  168
Socioeconomic intervention/rural  remapped cnt:  125
Ecosystem intervention/ecosystem  remapped cnt:  104
Socioeconomic intervention/government  remapped cnt:  253
Socioeconomic intervention/health  remapped cnt:  112
Socioeconomic intervention/social  remapped cnt:  162
Technology intervention/health  remapped cnt:  68
Technology intervention/energy  remapped cnt:  171
Technology intervention/genetic  remapped cnt:  232
Technology intervention/soil  remapped cnt:  37

In [113]:
for file in os.listdir(folder_with_processed):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_processed, file))
    df["interventions_found_before_mapping"] = ""
    for i in range(len(df)):
        new_interventions = []
        df["interventions_found_before_mapping"].values[i] = df["interventions_found"].values[i].copy()
        for interv in df["interventions_found_before_mapping"].values[i]:
            interv_split_words = interv.split("/")
            if len(interv_split_words) < 3 or not interv_split_words[2].strip():
                new_interventions.append(interv)
                continue
            raw_intervention = "/".join(interv_split_words[2:])
            intervention_group = "/".join(interv_split_words[:2])
            if intervention_group in remappings:
                if raw_intervention in remappings[intervention_group]:
                    new_interventions.append("/".join([intervention_group, remappings[intervention_group][raw_intervention]]))
        df["interventions_found"].values[i] = list(set(new_interventions))
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_with_processed, file))

Read file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 24.04s
Processed file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 57.88s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2000.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 21.69s
Processed file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 47.50s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2001.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 21.43s
Processed file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 54.58s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2002.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 23.45s
Processed file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 51.02s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2003.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2004.xlsx: 23.76s
Processed file ../tmp/processed_cosai_all_data\data_year_2004.xlsx

## CABI codes and sectors mapping

In [10]:
from text_processing import search_engine_insensitive_to_spelling
from text_processing import all_column_filler

cabi_codes_mapping = {}
cabi_codes_mapping_df = excel_reader.ExcelReader().read_df_from_excel("../data/cabi_codes_mapping.xlsx")
for i in range(len(cabi_codes_mapping_df)):
    if cabi_codes_mapping_df["CABI_CODE_NAME"].values[i].strip():
        cabi_codes_mapping[cabi_codes_mapping_df["CABI_CODE"].values[i].strip()] = cabi_codes_mapping_df["CABI_CODE_NAME"].values[i].strip()

sectors_mapping = {}
df_sectors = excel_reader.ExcelReader().read_df_from_excel("../data/sectors_mapping.xlsx")
for i in range(len(df_sectors)):
    sectors_mapping[df_sectors["Sector"].values[i]] = {
        "cabi_code_names": df_sectors["cabi_code_names"].values[i].split(";"),
        "keywords_to_find": list(filter(lambda x: x.strip(), df_sectors["keywords_to_find"].values[i].split(";")))
    }

for file in os.listdir(folder_with_processed):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_processed, file))
    search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
        load_abbreviations = True)
    file_part = file.split(".")[0]
    if os.path.exists(folder_with_search_indices + file_part):
        search_engine_inverted_index.load_model(folder_with_search_indices + file_part)
    else:
        search_engine_inverted_index.create_inverted_index(df)
        search_engine_inverted_index.save_model(folder_with_search_indices + file_part)

    df["cabi_codes"] = ""
    for i in range(len(df)):
        df["cabi_codes"].values[i] = []
        for val in df["cabi_subjects"].values[i].split(";"):
            if val.upper() in cabi_codes_mapping:
                df["cabi_codes"].values[i].append(cabi_codes_mapping[val.upper()])

    sector_doc_keywords_mapping = {}
    for sector in sectors_mapping:
        sector_doc_keywords_mapping[sector] = set()
        for key in sectors_mapping[sector]["keywords_to_find"]:
            sector_doc_keywords_mapping[sector].update(search_engine_inverted_index.find_articles_with_keywords([key]))

    df["Sector"] = ""
    for i in range(len(df)):
        df["Sector"].values[i] = set()
        for sector in sectors_mapping:
            cabi_code_found = False
            for cabi_code in sectors_mapping[sector]["cabi_code_names"]:
                if cabi_code in df["cabi_codes"].values[i]:
                    cabi_code_found = True
                    break
            if not cabi_code_found:
                continue
            if sectors_mapping[sector]["keywords_to_find"]:
                if i in sector_doc_keywords_mapping[sector]:
                    df["Sector"].values[i].add(sector)
            else:
                df["Sector"].values[i].add(sector)
        df["Sector"].values[i] = list(df["Sector"].values[i])
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_with_processed, file))

In [45]:
keywords_to_map = {
    "wheat": "wheat/cereals",
    "fruits": "fruits and vegetables",
    "vegetables": "fruits and vegetables",
    "cereals": "wheat/cereals",
    "buckwheat": "wheat/cereals",
    "root crops": "roots, tubers and bananas",
    "root vegetables": "roots, tubers and bananas",
    "bananas": "roots, tubers and bananas",
    "cassava": "roots, tubers and bananas",
    "potatoes": "roots, tubers and bananas",
    "sweet potatoes": "roots, tubers and bananas",
    "tuber": "roots, tubers and bananas"
}
folder_with_processed = "../tmp/processed_cosai_all_data"
for file in os.listdir(folder_with_processed):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_processed, file))
    for i in range(len(df)):
        additional_areas = set()
        for key_word in keywords_to_map:
            if key_word in df["plant_products_search"].values[i]:
                additional_areas.add(keywords_to_map[key_word])
        df["plant_products_search"].values[i].extend(list(additional_areas))
        df["plant_products_search"].values[i] = list(set(df["plant_products_search"].values[i]))
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_with_processed, file))

Read file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 34.23s
Processed file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 51.49s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2000.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 16.79s
Processed file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 39.93s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2001.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 17.93s
Processed file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 47.61s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2002.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 18.12s
Processed file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 47.06s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2003.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2004.xlsx: 28.33s
Processed file ../tmp/processed_cosai_all_data\data_year_2004.xlsx

In [5]:
# fix female/male found in connection with crops
sex_population_df = excel_reader.ExcelReader().read_df_from_excel("../data/map_sex.xlsx")
sex_population_tags_map = {}
for i in range(len(sex_population_df)):
    if sex_population_df["High level label"].values[i] not in sex_population_tags_map:
        sex_population_tags_map[sex_population_df["High level label"].values[i]] = []
    sex_population_tags_map[sex_population_df["High level label"].values[i]].append(
        text_normalizer.get_stemmed_words_inverted_index(sex_population_df["Keyword"].values[i]))

def check_sex_connection_to_crops(found_crops, whole_text):
    all_sex_connected_to_crop = {}
    for label in sex_population_tags_map:
        all_sex_connected_to_crop[label] = {"to_crops": 0, "other": 0}
    for text in text_normalizer.normalize_sentences(text_normalizer.tokenize_words(whole_text)):
        prepared_text = text_normalizer.get_stemmed_words_inverted_index(" ".join(text))
        if not prepared_text:
            continue
        for idx, word in enumerate(prepared_text):
            start_ind_found = False
            found_label = ""
            found_keywords = []
            for label in sex_population_tags_map:
                for key_word_array in sex_population_tags_map[label]:
                    for key_word in key_word_array:
                        if word == key_word:
                            start_ind_found = True
                            found_label = label
                            found_keywords = key_word_array
                            break
            if start_ind_found:
                the_whole_keyword_match = True
                for key_i, key_word in enumerate(found_keywords):
                    if (idx + key_i) >= len(prepared_text) or key_word != prepared_text[idx + key_i]:
                        the_whole_keyword_match = False
                if the_whole_keyword_match:
                    words_idx_after_population_tag = idx + len(found_keywords)
                    found_crop_phrase = False
                    for offset in range(0, 2):
                        for phrase_length in range(1, 5):
                            cur_pos = offset + words_idx_after_population_tag
                            if " ".join(prepared_text[cur_pos: cur_pos + phrase_length]) in found_crops:
                                found_crop_phrase = True
                                #print(" ".join(prepared_text[cur_pos: cur_pos + phrase_length]))
                                #print(found_keywords)
                                break
                    if found_crop_phrase:
                        all_sex_connected_to_crop[found_label]["to_crops"] += 1
                    else:
                        all_sex_connected_to_crop[found_label]["other"] += 1
    return all_sex_connected_to_crop

folder_with_processed = "../tmp/processed_cosai_all_data"
folder_with_search_indices = "../model/cosai_index_deduplicated/"
cnt_changed = 0
for file in os.listdir(folder_with_processed):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_processed, file))
    for i in range(len(df)):
        if "Unspecified" in df["sex_mentioned"].values[i]:
            continue
        found_crops = set(["cypress"])
        for column in ["plant_products_search_details", "animal_products_search_details", "animals_found_details"]:
            for obj in df[column].values[i]:
                found_crops.add(" ".join(text_normalizer.get_stemmed_words_inverted_index(obj)))
        whole_text = df["title"].values[i] + "." + df["abstract"].values[i]
        all_sex_connected_to_crop = check_sex_connection_to_crops(found_crops, whole_text)
        sex_mentioned_new = []
        for label in all_sex_connected_to_crop:
            if all_sex_connected_to_crop[label]["to_crops"] and not all_sex_connected_to_crop[label]["other"]:
                cnt_changed += 1
                continue
            if label in df["sex_mentioned"].values[i]:
                sex_mentioned_new.append(label)
        if not sex_mentioned_new:
            sex_mentioned_new = ["Unspecified"]
        #if sex_mentioned_new != df["sex_mentioned"].values[i]:
        #    print(i, sex_mentioned_new, df["sex_mentioned"].values[i], all_sex_connected_to_crop)
        df["sex_mentioned"].values[i] = sex_mentioned_new
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_with_processed, file))
    print("Number of changed articles ", cnt_changed)

Read file ../data/map_sex.xlsx: 0.01s
Processed file ../data/map_sex.xlsx: 0.00s
Read file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 21.64s
Processed file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 54.64s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2000.xlsx
Number of changed articles  168
Read file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 18.92s
Processed file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 39.25s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2001.xlsx
Number of changed articles  311
Read file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 17.47s
Processed file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 41.05s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2002.xlsx
Number of changed articles  505
Read file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 19.56s
Processed file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 42.59s
Saving...
Saved to ../tmp/proc

In [8]:
acknowledgements_data_ids = set()
for file in os.listdir(r"C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full"):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(r"C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full", file))
    for i in range(len(df)):
        if df["acknowledgement_part"].values[i].strip():
            acknowledgements_data_ids.add(int(df["pdf_file"].values[i].split(".")[0]))
len(acknowledgements_data_ids)

Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\0.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\0.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\1.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\1.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciH

Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\12.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\12.xlsx: 0.01s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\120.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\120.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-usin

Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\14.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\14.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\140.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\140.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-usin

Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\16.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\160.xlsx: 0.03s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\160.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\161.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DO

Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\18.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\180.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\180.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\181.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DO

...

Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\468.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\468.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\469.xlsx: 0.01s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\469.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-us

Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\488.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\489.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\489.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\49.xlsx: 0.03s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DO

Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\64.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\64.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\65.xlsx: 0.01s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\65.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-

Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\88.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\89.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\89.xlsx: 0.00s
Read file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full\9.xlsx: 0.02s
Processed file C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-us

33359

In [11]:
regions_found = {}
for file in os.listdir(folder_with_processed):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_processed, file))
    for i in range(len(df)):
        if df["id"].values[i] in acknowledgements_data_ids:
            for region in df["geo_regions"].values[i]:
                if region not in regions_found:
                    regions_found[region] = 0
                regions_found[region] += 1
regions_found

Read file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 26.59s
Processed file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 55.77s
Read file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 17.91s
Processed file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 38.77s
Read file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 19.05s
Processed file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 48.74s
Read file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 21.64s
Processed file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 47.42s
Read file ../tmp/processed_cosai_all_data\data_year_2004.xlsx: 21.81s
Processed file ../tmp/processed_cosai_all_data\data_year_2004.xlsx: 55.95s
Read file ../tmp/processed_cosai_all_data\data_year_2005.xlsx: 24.88s
Processed file ../tmp/processed_cosai_all_data\data_year_2005.xlsx: 58.93s
Read file ../tmp/processed_cosai_all_data\data_year_2006.xlsx: 26.81s
Processed file ../tmp/processed_cosai_all_data\data_year_200

{'South Asia': 5057,
 'Sub-Saharan Africa': 10420,
 'East Asia and Pacific': 18193,
 'North America': 5240,
 'Middle East and North Africa': 2194,
 'Latin America and the Caribbean': 1585,
 'Europe': 2766,
 'Central Asia': 474}

In [12]:
for region, cnt in sorted(regions_found.items(), key=lambda x: x[1], reverse=True):
    print(region, cnt)

East Asia and Pacific 18193
Sub-Saharan Africa 10420
North America 5240
South Asia 5057
Europe 2766
Middle East and North Africa 2194
Latin America and the Caribbean 1585
Central Asia 474


# Acknowledgements data extraction

In [17]:
def normalize_text(text):
    text = re.sub("-\s*\n", "", text)
    text = re.sub("\s+", " ",text.replace("\n", " "))
    text = re.sub("\(cid:\d+\)","", text)
    return text

In [ ]:
ner_org_model = spacy.load("../model/ner-org-model-13_iter/")

In [ ]:
folder = r"C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\folder_with_ack_full"
folder_to_save = "../tmp/acknowledgements_data"
os.makedirs(folder_to_save, exist_ok=True)
for file in os.listdir(folder):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder, file))
    df["extracted_organizations_custom"] = ""
    df["extracted_organizations_spacy"] = ""
    for i in range(len(df)):
        if df["acknowledgement_part"].values[i]:
            prepared_text = normalize_text(df["acknowledgement_part"].values[i])
            organizations = []
            for _ent in ner_org_model(prepared_text).ents:
                if _ent.label_ == "ORG":
                    organizations.append(_ent.text)
            df["extracted_organizations_custom"].values[i] = ";".join(organizations)
            
            organizations_spacy = []
            for _ent in nlp(prepared_text).ents:
                if _ent.label_ in ["ORG", "PRODUCT"]:
                    organizations_spacy.append(_ent.text)
            df["extracted_organizations_spacy"].values[i] = ";".join(organizations_spacy)
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_to_save, file))

In [99]:
from text_processing import affiliations_extractor
affiliations_extractor = reload(affiliations_extractor)

_affiliation_extractor = affiliations_extractor.AffiliationsExtractor()
organizations_dict = {}
acknowledgement_data_by_articles = {}
folder_with_extracted = "../tmp/acknowledgements_data"
for file in os.listdir(folder_with_extracted):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_extracted, file))
    for i in range(len(df)):
        article_id = int(df["pdf_file"].values[i].split(".")[0])
        acknowledgement_data_by_articles[article_id] = []
        for column in ["extracted_organizations_custom", "extracted_organizations_spacy"]:
            for org in df[column].values[i].split(";"):
                if org.strip():
                    normalized_text = text_normalizer.normalize_sentence(org)
                    if len(normalized_text.replace(" ", "")) <= 2:
                        continue
                    found_acknowledge_keywords = False
                    for keyword in ["acknowledgment", "acknowledgement", "acknowledge"]:
                        if keyword in normalized_text.lower():
                            found_acknowledge_keywords = True
                            break
                    if found_acknowledge_keywords:
                        continue
                    org_name = re.sub("\d+", " ", org)
                    org_name = re.sub("[^\w &]", " ", org_name)
                    org_name = " ".join([word for word in org_name.split() if not re.search(r"\b(the|a)\b", word, flags=re.IGNORECASE)])
                    org_name = text_normalizer.to_camel_case(_affiliation_extractor.fix_clipping(
                                org_name))
                    org_name = re.sub(r"^(and\b|\&)", "", org_name).strip()
                    org_name = re.sub("\s+", " ", org_name).strip()
                    org_name = _affiliation_extractor.merge_one_letter_abbreviations(org_name)
                    if len(org_name.replace(" ", "")) <= 2:
                        continue
                    acknowledgement_data_by_articles[article_id].append(normalized_text)
                    if normalized_text not in organizations_dict:
                        organizations_dict[normalized_text] = {}
                    if org_name not in organizations_dict[normalized_text]:
                        organizations_dict[normalized_text][org_name] = 0
                    organizations_dict[normalized_text][org_name] += 1

Read file ../data/map_clipped_words.xlsx: 0.01s
Processed file ../data/map_clipped_words.xlsx: 0.00s
Read file ../data/countries_affiliation_remapping.xlsx: 0.01s
Processed file ../data/countries_affiliation_remapping.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\0.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\0.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\1.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\1.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\10.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\10.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\100.xlsx: 0.03s
Processed file ../tmp/acknowledgements_data\100.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\101.xlsx: 0.03s
Processed file ../tmp/acknowledgements_data\101.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\102.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\102.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\103.xlsx: 0.02s
Processed file ../tmp/acknowledgeme

Read file ../tmp/acknowledgements_data\161.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\161.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\162.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\162.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\163.xlsx: 0.01s
Processed file ../tmp/acknowledgements_data\163.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\164.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\164.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\165.xlsx: 0.01s
Processed file ../tmp/acknowledgements_data\165.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\166.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\166.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\167.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\167.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\168.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\168.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\169.xlsx: 0.02s
Processed file ../tmp/ack

Read file ../tmp/acknowledgements_data\226.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\226.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\227.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\227.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\228.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\228.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\229.xlsx: 0.01s
Processed file ../tmp/acknowledgements_data\229.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\23.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\23.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\230.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\230.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\231.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\231.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\232.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\232.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\233.xlsx: 0.02s
Processed file ../tmp/ackno

Read file ../tmp/acknowledgements_data\291.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\291.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\292.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\292.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\293.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\293.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\294.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\294.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\295.xlsx: 0.03s
Processed file ../tmp/acknowledgements_data\295.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\296.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\296.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\297.xlsx: 0.03s
Processed file ../tmp/acknowledgements_data\297.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\298.xlsx: 0.03s
Processed file ../tmp/acknowledgements_data\298.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\299.xlsx: 0.02s
Processed file ../tmp/ack

Read file ../tmp/acknowledgements_data\356.xlsx: 0.01s
Processed file ../tmp/acknowledgements_data\356.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\357.xlsx: 0.01s
Processed file ../tmp/acknowledgements_data\357.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\358.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\358.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\359.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\359.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\36.xlsx: 0.01s
Processed file ../tmp/acknowledgements_data\36.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\360.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\360.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\361.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\361.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\362.xlsx: 0.03s
Processed file ../tmp/acknowledgements_data\362.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\363.xlsx: 0.02s
Processed file ../tmp/ackno

Read file ../tmp/acknowledgements_data\420.xlsx: 0.01s
Processed file ../tmp/acknowledgements_data\420.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\421.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\421.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\422.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\422.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\423.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\423.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\424.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\424.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\425.xlsx: 0.03s
Processed file ../tmp/acknowledgements_data\425.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\426.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\426.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\427.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\427.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\428.xlsx: 0.02s
Processed file ../tmp/ack

Read file ../tmp/acknowledgements_data\486.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\486.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\487.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\487.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\488.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\488.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\489.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\489.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\49.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\49.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\490.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\490.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\491.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\491.xlsx: 0.01s
Read file ../tmp/acknowledgements_data\492.xlsx: 0.02s
Processed file ../tmp/acknowledgements_data\492.xlsx: 0.00s
Read file ../tmp/acknowledgements_data\493.xlsx: 0.02s
Processed file ../tmp/ackno

In [100]:
items_to_save = []
items_counts = []
organization_name_by_normalized = {}
for org in organizations_dict:
    all_cnt = 0
    max_cnt = 0
    frequent_name = ""
    for org_name, cnt in organizations_dict[org].items():
        all_cnt += cnt
        if max_cnt < cnt:
            max_cnt = cnt
            frequent_name = org_name
    items_to_save.append((org, frequent_name, all_cnt, max_cnt))
    items_counts.append((frequent_name, all_cnt))
    organization_name_by_normalized[org] = frequent_name
excel_writer.ExcelWriter().save_data_in_excel(items_to_save, ["Normalized name", "Org name", "Frequency", "Max org name freq"], "org_names_freq.xlsx")
folder_with_orgs = "../tmp/folder_with_orgs"
os.makedirs(folder_with_orgs, exist_ok=True)
excel_writer.ExcelWriter().save_data_in_excel(items_counts, ["affiliation", "count"], os.path.join(
    folder_with_orgs, "org_names_count.xlsx"))

Saving...
Saved to org_names_freq.xlsx
Saving...
Saved to ../tmp/folder_with_orgs\org_names_count.xlsx


In [101]:
_affiliation_extractor.univ_raw_words_remapped = _affiliation_extractor.create_dictionary_for_univ_words(
    folder_with_orgs)

Read file ../tmp/folder_with_orgs\org_names_count.xlsx: 8.17s
Processed file ../tmp/folder_with_orgs\org_names_count.xlsx: 5.73s


In [103]:
folder_with_processed = "../tmp/processed_cosai_all_data"
for file in os.listdir(folder_with_processed):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_processed, file))
    df["acknowledgement_org_names"] = ""
    for i in range(len(df)):
        if df["id"].values[i] in acknowledgement_data_by_articles:
            org_names = []
            for org in acknowledgement_data_by_articles[df["id"].values[i]]:
                org_names.append(organization_name_by_normalized[org])
            df["acknowledgement_org_names"].values[i] = list(set(org_names))
            if not org_names:
                df["acknowledgement_org_names"].values[i] = ["Unspecified"]
        else:
            df["acknowledgement_org_names"].values[i] = []
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_with_processed, file))

Read file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 36.18s
Processed file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 63.56s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2000.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 20.70s
Processed file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 48.33s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2001.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 24.13s
Processed file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 55.09s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2002.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 24.86s
Processed file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 59.24s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2003.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2004.xlsx: 28.13s
Processed file ../tmp/processed_cosai_all_data\data_year_2004.xlsx

In [3]:
dict_with_orgs_remapped = {}
folder_with_funders_manual = "../tmp/funders_data_manual_checked/"
for file in os.listdir(folder_with_funders_manual):
    df_mapped = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_funders_manual, file))
    for i in range(len(df_mapped)):
        if df_mapped["Funder"].values[i] == 1:
            dict_with_orgs_remapped[df_mapped["raw_affiliation"].values[i]] = df_mapped["mapped_affiliation"].values[i]

Read file ../tmp/funders_data_manual_checked/orgs_to_check_manual_Jaron.xlsx: 0.43s
Processed file ../tmp/funders_data_manual_checked/orgs_to_check_manual_Jaron.xlsx: 0.19s
Read file ../tmp/funders_data_manual_checked/orgs_to_check_manual_Sarah (1).xlsx: 0.18s
Processed file ../tmp/funders_data_manual_checked/orgs_to_check_manual_Sarah (1).xlsx: 0.15s
Read file ../tmp/funders_data_manual_checked/orgs_to_check_manual_Tom.xlsx: 0.19s
Processed file ../tmp/funders_data_manual_checked/orgs_to_check_manual_Tom.xlsx: 0.16s


In [4]:
len(dict_with_orgs_remapped)

1833

In [5]:
folder_with_processed = "../tmp/processed_cosai_all_data"
for file in os.listdir(folder_with_processed):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_processed, file))
    for i in range(len(df)):
        org_names = []
        for org in df["acknowledgement_org_names"].values[i]:
            if org == "Unspecified":
                org_names.append("Unspecified")
            elif org in dict_with_orgs_remapped:
                org_names.append(dict_with_orgs_remapped[org])
        if not org_names and df["acknowledgement_org_names"].values[i]:
            org_names = ["Unspecified"]
        df["acknowledgement_org_names"].values[i] = list(set(org_names))
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_with_processed, file))

Read file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 12.79s
Processed file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 43.84s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2000.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 13.66s
Processed file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 31.23s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2001.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 16.51s
Processed file ../tmp/processed_cosai_all_data\data_year_2002.xlsx: 40.84s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2002.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 16.66s
Processed file ../tmp/processed_cosai_all_data\data_year_2003.xlsx: 39.17s
Saving...
Saved to ../tmp/processed_cosai_all_data\data_year_2003.xlsx
Read file ../tmp/processed_cosai_all_data\data_year_2004.xlsx: 23.15s
Processed file ../tmp/processed_cosai_all_data\data_year_2004.xlsx

# Prepare ids and other fields

In [7]:
#prepare ids and other fields
global_id = 0
cnt = 0
for file in os.listdir(folder_with_processed):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_processed, file))
    cnt += len(df)
    df["id"] = 0
    df["comments"] = ""
    df["tags"] = [[] for i in range(len(df))]
    df["edit_url"] = ""
    for j in range(len(df)):
        df["id"].values[j] = global_id
        global_id += 1
        df["edit_url"].values[j] = str(df["id"].values[j])
    
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_with_processed, file))

# Processing pdfs for finding funding sources

In [11]:
cnt_all = 0
cnt_with_urls = 0
for file in os.listdir("../tmp/processed_deduplicated_cosai_processed"):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/processed_deduplicated_cosai_processed", file))
    cnt_all += len(df)
    for i in range(len(df)):
        if df["url"].values[i].strip():
            cnt_with_urls += 1
cnt_all, cnt_with_urls

Read file ../tmp/processed_deduplicated_cosai_processed\data_year_2000.xlsx: 12.13s
Processed file ../tmp/processed_deduplicated_cosai_processed\data_year_2000.xlsx: 27.53s
Read file ../tmp/processed_deduplicated_cosai_processed\data_year_2001.xlsx: 9.69s
Processed file ../tmp/processed_deduplicated_cosai_processed\data_year_2001.xlsx: 22.47s
Read file ../tmp/processed_deduplicated_cosai_processed\data_year_2002.xlsx: 11.87s
Processed file ../tmp/processed_deduplicated_cosai_processed\data_year_2002.xlsx: 24.80s
Read file ../tmp/processed_deduplicated_cosai_processed\data_year_2003.xlsx: 12.06s
Processed file ../tmp/processed_deduplicated_cosai_processed\data_year_2003.xlsx: 27.91s
Read file ../tmp/processed_deduplicated_cosai_processed\data_year_2004.xlsx: 11.31s
Processed file ../tmp/processed_deduplicated_cosai_processed\data_year_2004.xlsx: 25.67s
Read file ../tmp/processed_deduplicated_cosai_processed\data_year_2005.xlsx: 13.23s
Processed file ../tmp/processed_deduplicated_cosai_p

(1221921, 645068)

In [20]:
choose_ids_for_urls_finding_criteria_first = set()
choose_ids_for_urls_finding_criteria_second = set()
ids_by_type = {}
others_by_year = {}
ids_with_urls = {}
for file in os.listdir("../tmp/processed_cosai_all_data/"):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(
        "../tmp/processed_cosai_all_data/", file))
    for i in range(len(df)):
        if not df["url"].values[i].strip():
            continue
        ids_with_urls[df["id"].values[i]] = df["url"].values[i]
        found_geo_regions = False
        for region in ["East Asia and Pacific", "Sub-Saharan Africa",
                       "Middle East and North Africa", "South Asia"]:
            if region in df["geo_regions"].values[i]:
                found_geo_regions = True
                break
        if not found_geo_regions:
            continue
        if "Small scale farmers" in df["farm_size_tags"].values[i]:
            choose_ids_for_urls_finding_criteria_first.add(df["id"].values[i])
        found_study_types = False
        for study_type in ["Landscape/environmental simulation", "Randomized controlled trials",
                          "Impact evaluation", "Field study", "Policy analysis", "Observational study"]:
            if study_type in df["study_type"].values[i]:
                found_study_types = True
                if study_type not in ids_by_type:
                    ids_by_type[study_type] = set()
                ids_by_type[study_type].add(df["id"].values[i])
        if found_study_types:
            choose_ids_for_urls_finding_criteria_second.add(df["id"].values[i])
        if df["year"].values[i] not in others_by_year:
            others_by_year[df["year"].values[i]] = set()
        others_by_year[df["year"].values[i]].add(df["id"].values[i])

Read file ../tmp/processed_cosai_all_data/data_year_2000.xlsx: 18.50s
Processed file ../tmp/processed_cosai_all_data/data_year_2000.xlsx: 48.87s
Read file ../tmp/processed_cosai_all_data/data_year_2001.xlsx: 14.84s
Processed file ../tmp/processed_cosai_all_data/data_year_2001.xlsx: 36.01s
Read file ../tmp/processed_cosai_all_data/data_year_2002.xlsx: 18.37s
Processed file ../tmp/processed_cosai_all_data/data_year_2002.xlsx: 39.06s
Read file ../tmp/processed_cosai_all_data/data_year_2003.xlsx: 19.22s
Processed file ../tmp/processed_cosai_all_data/data_year_2003.xlsx: 41.31s
Read file ../tmp/processed_cosai_all_data/data_year_2004.xlsx: 19.02s
Processed file ../tmp/processed_cosai_all_data/data_year_2004.xlsx: 46.07s
Read file ../tmp/processed_cosai_all_data/data_year_2005.xlsx: 21.35s
Processed file ../tmp/processed_cosai_all_data/data_year_2005.xlsx: 50.73s
Read file ../tmp/processed_cosai_all_data/data_year_2006.xlsx: 22.11s
Processed file ../tmp/processed_cosai_all_data/data_year_200

In [21]:
all_ids = set()
all_ids = all_ids.union(choose_ids_for_urls_finding_criteria_first)
for study_type in ["Randomized controlled trials",
                   "Impact evaluation", "Policy analysis"]:
    all_ids = all_ids.union(ids_by_type[study_type])
len(all_ids)

14630

In [27]:
for year in others_by_year:
    land_num = 0
    field_num = 0
    observ = 0
    for _id in others_by_year[year]:
        if _id in all_ids:
            continue
        if _id in ids_by_type["Landscape/environmental simulation"] and land_num < 400:
            land_num += 1
            all_ids.add(_id)
        elif _id in ids_by_type["Field study"] and field_num < 400:
            field_num += 1
            all_ids.add(_id)
        elif _id in ids_by_type["Observational study"] and observ < 800:
            observ += 1
            all_ids.add(_id)
len(all_ids)

46040

In [28]:
others = list(choose_ids_for_urls_finding_criteria_second - all_ids)
np.random.shuffle(others)

In [29]:
num_to_take = 50000 - len(all_ids)
len(others[:num_to_take])

3960

In [30]:
all_ids = all_ids.union(set(others[:num_to_take]))

In [31]:
urls_to_use = []
for _id in sorted(all_ids):
    urls_to_use.append((_id, ids_with_urls[_id]))
len(urls_to_use)

50000

In [32]:
df = pd.DataFrame(urls_to_use, columns=["id", "url"])

In [38]:
df.head()

,id,url,title
0,19,https://www.doi.org/10.1016/S0196-8904(99)00095-3,
1,22,https://www.doi.org/10.1016/S0309-1740(99)00078-9,
2,28,https://www.doi.org/10.1023/A:1005241002743,
3,53,https://www.doi.org/10.1046/j.1365-2400.2000.0...,
4,55,https://www.doi.org/10.1016/S0044-8486(99)00294-X,


In [41]:
titles_dict = {}
for file in os.listdir("../tmp/processed_cosai_all_data/"):
    df_1 = excel_reader.ExcelReader().read_df_from_excel(os.path.join(
        "../tmp/processed_cosai_all_data/", file))
    for i in range(len(df_1)):
        titles_dict[df_1["id"].values[i]] = df_1["title"].values[i]

Read file ../tmp/processed_cosai_all_data/data_year_2000.xlsx: 20.58s
Processed file ../tmp/processed_cosai_all_data/data_year_2000.xlsx: 51.73s
Read file ../tmp/processed_cosai_all_data/data_year_2001.xlsx: 16.39s
Processed file ../tmp/processed_cosai_all_data/data_year_2001.xlsx: 39.59s
Read file ../tmp/processed_cosai_all_data/data_year_2002.xlsx: 18.59s
Processed file ../tmp/processed_cosai_all_data/data_year_2002.xlsx: 48.99s
Read file ../tmp/processed_cosai_all_data/data_year_2003.xlsx: 19.56s
Processed file ../tmp/processed_cosai_all_data/data_year_2003.xlsx: 46.81s
Read file ../tmp/processed_cosai_all_data/data_year_2004.xlsx: 16.78s
Processed file ../tmp/processed_cosai_all_data/data_year_2004.xlsx: 52.15s
Read file ../tmp/processed_cosai_all_data/data_year_2005.xlsx: 19.50s
Processed file ../tmp/processed_cosai_all_data/data_year_2005.xlsx: 51.43s
Read file ../tmp/processed_cosai_all_data/data_year_2006.xlsx: 20.93s
Processed file ../tmp/processed_cosai_all_data/data_year_200

In [42]:
df["title"] = ""
for i in range(len(df)):
    df["title"].values[i] = titles_dict[df["id"].values[i]]

In [43]:
excel_writer.ExcelWriter().save_big_data_df_in_excel(df, "../tmp/urls_to_find_funders_1", portion_number=100)

Saving...
Saved to ../tmp/urls_to_find_funders_1\0.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\1.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\2.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\3.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\4.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\5.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\6.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\7.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\8.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\9.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\10.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\11.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\12.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\13.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\14.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\15.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\16.xlsx
Saving...
Saved to ../tmp/urls_to_find_fu

Saving...
Saved to ../tmp/urls_to_find_funders_1\147.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\148.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\149.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\150.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\151.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\152.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\153.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\154.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\155.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\156.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\157.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\158.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\159.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\160.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\161.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\162.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\163.xlsx
Saving...
Save

Saving...
Saved to ../tmp/urls_to_find_funders_1\289.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\290.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\291.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\292.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\293.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\294.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\295.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\296.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\297.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\298.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\299.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\300.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\301.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\302.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\303.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\304.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\305.xlsx
Saving...
Save

Saved to ../tmp/urls_to_find_funders_1\434.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\435.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\436.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\437.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\438.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\439.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\440.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\441.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\442.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\443.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\444.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\445.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\446.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\447.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\448.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\449.xlsx
Saving...
Saved to ../tmp/urls_to_find_funders_1\450.xlsx
Saving...
Saved to ../tm

In [16]:
len(choose_ids_for_urls_finding_criteria_second - choose_ids_for_urls_finding_criteria_first)

109244

In [19]:
for study_type in ids_by_type:
    print(study_type, len([_id for _id in ids_by_type[study_type] if _id not in choose_ids_for_urls_finding_criteria_first]))

Field study 31790
Observational study 56178
Landscape/environmental simulation 27490
Randomized controlled trials 1368
Impact evaluation 2701
Policy analysis 218


In [26]:
for year in others_by_year:
    print(year, len(others_by_year[year]))

2000 2046
2001 1805
2002 2007
2003 2356
2004 2579
2005 3087
2006 3413
2007 4170
2008 4793
2009 5872
2010 6710
2011 8294
2012 8877
2013 9199
2014 10137
2015 10115
2016 10805
2017 18322
2018 21661
2019 24772
2020 26990
2021 8784


In [53]:
df = excel_reader.ExcelReader().read_df_from_excel("../tmp/organizations_extraction_data/Mary_funders_info.xlsx")

Read file ../tmp/organizations_extraction_data/Mary_funders_info.xlsx: 0.24s
Processed file ../tmp/organizations_extraction_data/Mary_funders_info.xlsx: 0.01s


In [54]:
for i in range(len(df)):
    df["Acknowledgement Data"].values[i] = df["Acknowledgement Data"].values[i].replace("\n", " ")

In [55]:
excel_writer.ExcelWriter().save_df_in_excel(df, "../tmp/organizations_extraction_data/Mary_funders_info.xlsx")

Saving...
Saved to ../tmp/organizations_extraction_data/Mary_funders_info.xlsx


# Upload data to the dashboard

In [6]:
already_indexed = set()
drop_index = True
for file in ["data_year_2020.xlsx", "data_year_2019.xlsx"] + os.listdir(folder_with_processed):
    if file in already_indexed:
        continue
    already_indexed.add(file)
    print("Drop index ", drop_index)
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_with_processed, file))
    elastic.ElasticClient("ec2-54-69-80-159.us-west-2.compute.amazonaws.com:9200",
                          "ec2-54-69-80-159.us-west-2.compute.amazonaws.com:5601",
                          "elastic", "elastic123456havos").update_docs(
        df,
        properties={
            'title':      {'type': 'text'},
            'abstract':   {'type': 'text'},
            'year':       {'type': 'date'},
            'journal_name': {'type':'keyword'},
            'publisher': {'type':'keyword'},
            'url': {'type':'keyword'},
            'article_id': {'type':'keyword'},
            'normalized_key_words': {'type':'keyword'},
            'countries_mentioned': {'type':'keyword'},
            'country_codes': {'type':'keyword'},
            'provinces': {'type':'keyword'},
            'districts': {'type':'keyword'},
            'geo_regions': {'type':'keyword'},
            'world_bankdivision_regions': {'type':'keyword'},
            'plant_products_search': {'type':'keyword'},
            'animal_products_search': {'type':'keyword'},
            'animals_found': {'type':'keyword'},
            'plant_diseases_found': {'type':'keyword'},
            'animal_diseases_found': {'type':'keyword'},
            'pests_found': {'type':'keyword'},
            'pesticides_found': {'type':'keyword'},
            "interventions_found": {"type": "keyword"},
            'interventions_found_raw':{'type':'keyword'},
            'outcomes_found':{'type':'keyword'},
            'outcomes_details':{'type':'text'},
            "cabi_codes": {"type": "keyword"},
            "Sector": {"type": "keyword"},
            'study_type':{'type':'keyword'},
            'affiliation_processed':{'type':'keyword'},
            "authors_processed":{"type":"keyword"},
            "corporate_author_processed":{"type":"keyword"},
            "raw_authors":{"type":"text"},
            "raw_affiliation":{"type":"text"},
            "raw_corporate_author":{"type": "text"},
            "edit_url":{"type":"keyword"},
            "comments": {"type": "text"},
            "tags": {"type": "keyword"},
            "farm_size_tags": {"type": "keyword"},
            "ecological_zones": {"type": "keyword"},
            "sex_mentioned": {"type": "keyword"},
            "age_mentioned": {"type": "keyword"},
            "equity_mentioned": {"type": "keyword"},
            "agricultural_workers_mentioned": {"type": "keyword"},
            "acknowledgement_org_names": {"type": "keyword"}
        },
        index_name='cosai_docs',
        time_field='year',
        create_search=False,
        drop_index=drop_index,
        vers_7=False,
        field_for_url_with_template="edit_url",
        fields_for_url=["url"],
        webservice_edit_host="ec2-54-69-80-159.us-west-2.compute.amazonaws.com:8501"
    )
    if drop_index:
        drop_index = False

Drop index  True
Read file ../tmp/processed_cosai_all_data\data_year_2020.xlsx: 47.55s
Processed file ../tmp/processed_cosai_all_data\data_year_2020.xlsx: 130.33s
Creating 'cosai_docs' Elasticsearch index
Preparing bulk operation
Indexing 89385 docs...
Creating 'cosai_docs' Kibana index pattern object
Done
Drop index  False
Read file ../tmp/processed_cosai_all_data\data_year_2019.xlsx: 52.25s
Processed file ../tmp/processed_cosai_all_data\data_year_2019.xlsx: 194.09s
Preparing bulk operation
Indexing 108266 docs...
Creating 'cosai_docs' Kibana index pattern object
Done
Drop index  False
Read file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 15.35s
Processed file ../tmp/processed_cosai_all_data\data_year_2000.xlsx: 40.13s
Preparing bulk operation
Indexing 31806 docs...
Creating 'cosai_docs' Kibana index pattern object
Done
Drop index  False
Read file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 10.08s
Processed file ../tmp/processed_cosai_all_data\data_year_2001.xlsx: 24